## step1
한 이미지에서 csv,
4개의 object추출하니까 4개의 좌표값 가져오기, 
그 좌표값에 해당하는 feature가져와서 

test csv 다시 확장해서 -> 열 29개 추가하기.. 좋은데? 셔플을 하고 train_loader는 셔플하지말기

binary 모델 가져온거에다가 학습하기..

## step2
binary shap 해보기 


## step3
full모델에서 attention 들고올 수 있는지 확인 
nlp모델에서 attention 값을 가져온다..  

## step4
ui에서 유의미한걸 불러와야하는데 
정답률이 높은거랑 낮은걸 불러와야함.. 
그거랑 실제로 생성한거랑 정답 비교할 수 있어야함.. csv에다가 report랑 성능도 넣는거 있어야할듯 

## step5

UI 구성 어떻게 할지?

환자 ID정보 , 라벨 정보, csv에 있는 정보랑
ob모델이랑 nlp모델 시각화해주고 




# object detector

In [7]:
from ast import literal_eval
from copy import deepcopy
import logging
import os
import random
from typing import List, Dict

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from torch.utils.tensorboard import SummaryWriter
import torch
from torch import Tensor
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
from tqdm import tqdm
import sys 


In [8]:
current_dir = os.getcwd()  # 현재 스크립트의 디렉토리
parent_dir = os.path.abspath(os.path.join(current_dir, "../"))  # src 상위 디렉토리
sys.path.append(parent_dir)
from dataset.constants import ANATOMICAL_REGIONS# src.dataset.constants 였는데 바꿈
from object_detector.custom_image_dataset_object_detector import CustomImageDataset
from object_detector.object_detector import ObjectDetector
from path_datasets_and_weights import path_full_dataset, path_runs_object_detector

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [10]:
logging.basicConfig(level=logging.INFO, format="[%(levelname)s]: %(message)s")
log = logging.getLogger(__name__)

# define configurations for training run
RUN = 14
# comment can be useful to add additional information to run_config.txt file
RUN_COMMENT = "first_run""""Enter comment here."""
SEED = 41
IMAGE_INPUT_SIZE = 512
PERCENTAGE_OF_TRAIN_SET_TO_USE = 1.0
PERCENTAGE_OF_VAL_SET_TO_USE = 0.2
BATCH_SIZE = 16
EFFECTIVE_BATCH_SIZE = 64
NUM_WORKERS = 8
EPOCHS = 20
LR = 1e-3
EVALUATE_EVERY_K_STEPS = 1000  # how often to evaluate the model on the validation set and log metrics to tensorboard (additionally, model will always be evaluated at end of epoch)
PATIENCE_LR_SCHEDULER = 5  # number of evaluations to wait for val loss to reduce before lr is reduced
THRESHOLD_LR_SCHEDULER = 1e-3
FACTOR_LR_SCHEDULER = 0.5
COOLDOWN_LR_SCHEDULER = 5

# set the seed value for reproducibility
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)


def get_title(region_set, region_indices, region_colors, class_detected_img):
    # region_set always contains 6 region names (except for region_set_5)

    # get a list of 6 boolean values that specify if that region was detected
    class_detected = [class_detected_img[region_index] for region_index in region_indices]

    # add color_code to region name (e.g. "(r)" for red)
    # also add nd to the brackets if region was not detected (e.g. "(r, nd)" if red region was not detected)
    region_set = [region + f" ({color})" if cls_detect else region + f" ({color}, nd)" for region, color, cls_detect in zip(region_set, region_colors, class_detected)]

    # add a line break to the title, as to not make it too long
    return ", ".join(region_set[:3]) + "\n" + ", ".join(region_set[3:])


def plot_box(box, ax, clr, linestyle, class_detected=True):
    x0, y0, x1, y1 = box
    h = y1 - y0
    w = x1 - x0
    ax.add_artist(
        plt.Rectangle(
            xy=(x0, y0),
            height=h,
            width=w,
            fill=False,
            color=clr,
            linewidth=1,
            linestyle=linestyle
        )
    )

    # add an annotation to the gt box, that the pred box does not exist (i.e. the corresponding class was not detected)
    if not class_detected:
        ax.annotate("not detected", (x0, y0), color=clr, weight="bold", fontsize=10)


def plot_gt_and_pred_bboxes_to_tensorboard(writer, overall_steps_taken, images, detections, targets, class_detected, num_images_to_plot=2):
    # pred_boxes is of shape [batch_size x 29 x 4] and contains the predicted region boxes with the highest score (i.e. top-1)
    # they are sorted in the 2nd dimension, meaning the 1st of the 29 boxes corresponds to the 1st region/class,
    # the 2nd to the 2nd class and so on
    pred_boxes_batch = detections["top_region_boxes"]

    # targets is a list of dicts, with each dict containing the key "boxes" that contain the gt boxes of a single image
    # gt_boxes is of shape [batch_size x 29 x 4]
    gt_boxes_batch = torch.stack([t["boxes"] for t in targets], dim=0)

    # plot 6 regions at a time, as to not overload the image with boxes (except for region_set_5, which has 5 regions)
    # the region_sets were chosen as to minimize overlap between the contained regions (i.e. better visibility)
    region_set_1 = ["right lung", "right costophrenic angle", "left lung", "left costophrenic angle", "cardiac silhouette", "spine"]
    region_set_2 = ["right upper lung zone", "right mid lung zone", "right lower lung zone", "left upper lung zone", "left mid lung zone", "left lower lung zone"]
    region_set_3 = ["right hilar structures", "right apical zone", "left hilar structures", "left apical zone", "right hemidiaphragm", "left hemidiaphragm"]
    region_set_4 = ["trachea", "right clavicle", "left clavicle", "aortic arch", "abdomen", "right atrium"]
    region_set_5 = ["mediastinum", "svc", "cavoatrial junction", "carina", "upper mediastinum"]

    regions_sets = [region_set_1, region_set_2, region_set_3, region_set_4, region_set_5]

    for num_img in range(num_images_to_plot):
        image = images[num_img].cpu().numpy().transpose(1, 2, 0)

        gt_boxes_img = gt_boxes_batch[num_img]
        pred_boxes_img = pred_boxes_batch[num_img]
        class_detected_img = class_detected[num_img].tolist()

        for num_region_set, region_set in enumerate(regions_sets):
            fig = plt.figure(figsize=(8, 8))
            ax = plt.gca()

            plt.imshow(image, cmap='gray')
            plt.axis('off')

            region_indices = [ANATOMICAL_REGIONS[region] for region in region_set]
            region_colors = ["b", "g", "r", "c", "m", "y"]

            if num_region_set == 4:
                region_colors.pop()

            for region_index, color in zip(region_indices, region_colors):
                box_gt = gt_boxes_img[region_index].tolist()
                box_pred = pred_boxes_img[region_index].tolist()
                box_class_detected = class_detected_img[region_index]

                plot_box(box_gt, ax, clr=color, linestyle="solid", class_detected=box_class_detected)

                # only plot predicted box if class was actually detected
                if box_class_detected:
                    plot_box(box_pred, ax, clr=color, linestyle="dashed")

            title = get_title(region_set, region_indices, region_colors, class_detected_img)
            ax.set_title(title)

            writer.add_figure(f"img_{num_img}_region_set_{num_region_set}", fig, overall_steps_taken)


def compute_box_area(box):
    """
    Calculate the area of a box given the 4 corner values.

    Args:
        box (Tensor[batch_size x 29 x 4])

    Returns:
        area (Tensor[batch_size x 29])
    """
    x0 = box[..., 0]
    y0 = box[..., 1]
    x1 = box[..., 2]
    y1 = box[..., 3]

    return (x1 - x0) * (y1 - y0)


def compute_intersection_and_union_area_per_class(detections, targets, class_detected):
    # pred_boxes is of shape [batch_size x 29 x 4] and contains the predicted region boxes with the highest score (i.e. top-1)
    # they are sorted in the 2nd dimension, meaning the 1st of the 29 boxes corresponds to the 1st region/class,
    # the 2nd to the 2nd class and so on
    pred_boxes = detections["top_region_boxes"]

    # targets is a list of dicts, with each dict containing the key "boxes" that contain the gt boxes of a single image
    # gt_boxes is of shape [batch_size x 29 x 4]
    gt_boxes = torch.stack([t["boxes"] for t in targets], dim=0)

    # below tensors are of shape [batch_size x 29]
    x0_max = torch.maximum(pred_boxes[..., 0], gt_boxes[..., 0])
    y0_max = torch.maximum(pred_boxes[..., 1], gt_boxes[..., 1])
    x1_min = torch.minimum(pred_boxes[..., 2], gt_boxes[..., 2])
    y1_min = torch.minimum(pred_boxes[..., 3], gt_boxes[..., 3])

    # intersection_boxes is of shape [batch_size x 29 x 4]
    intersection_boxes = torch.stack([x0_max, y0_max, x1_min, y1_min], dim=-1)

    # below tensors are of shape [batch_size x 29]
    intersection_area = compute_box_area(intersection_boxes)
    pred_area = compute_box_area(pred_boxes)
    gt_area = compute_box_area(gt_boxes)

    # if x0_max >= x1_min or y0_max >= y1_min, then there is no intersection
    valid_intersection = torch.logical_and(x0_max < x1_min, y0_max < y1_min)

    # also there is no intersection if the class was not detected by object detector
    valid_intersection = torch.logical_and(valid_intersection, class_detected)

    # set all non-valid intersection areas to 0
    intersection_area = torch.where(valid_intersection, intersection_area, torch.tensor(0, dtype=intersection_area.dtype, device=intersection_area.device))

    union_area = (pred_area + gt_area) - intersection_area

    # sum up the values along the batch dimension (the values will divided by each other later to get the averages)
    intersection_area = torch.sum(intersection_area, dim=0)
    union_area = torch.sum(union_area, dim=0)

    return intersection_area, union_area


def get_val_loss_and_other_metrics(model, val_dl, writer, overall_steps_taken):
    """
    Args:
        model (nn.Module): The input model to be evaluated.
        val_dl (torch.utils.data.Dataloder): The val dataloader to evaluate on.
        writer (tensorboardX.SummaryWriter.writer): Writer used to plot gt and predicted bboxes of first couple of image in val set
        overall_steps_taken: for tensorboard

    Returns:
        val_loss (float): val loss for val set
        avg_num_detected_classes_per_image (float): since it's possible that certain classes/regions of all 29 regions are not detected in an image,
        this metric counts how many classes are detected on average for an image. Ideally, this number should be 29.0
        avg_detections_per_class (list[float]): this metric counts how many times a class was detected in an image on average. E.g. if the value is 1.0,
        then the class was detected in all images of the val set
        avg_iou_per_class (list[float]): average IoU per class computed over all images in val set
    """
    # PyTorch implementation only return losses in train mode, and only detections in eval mode
    # see https://stackoverflow.com/questions/60339336/validation-loss-for-pytorch-faster-rcnn/65347721#65347721
    # my model is modified to return losses, detections and class_detected in eval mode
    # see forward method of object detector class for more information
    model.eval()

    val_loss = 0.0

    num_images = 0

    # tensor for accumulating the number of times a class is detected over all images (will be divided by num_images at the end of get average)
    sum_class_detected = torch.zeros(29, device=device)

    # tensor for accumulating the intersection area of each class (will be divided by union area of each class at the end of get the IoU for each class)
    sum_intersection_area_per_class = torch.zeros(29, device=device)

    # tensor for accumulating the union area of each class (will divide the intersection area of each class at the end of get the IoU for each class)
    sum_union_area_per_class = torch.zeros(29, device=device)

    with torch.no_grad():
        for batch_num, batch in tqdm(enumerate(val_dl)):
            # "targets" maps to a list of dicts, where each dict has the keys "boxes" and "labels" and corresponds to a single image
            # "boxes" maps to a tensor of shape [29 x 4] and "labels" maps to a tensor of shape [29]
            # note that the "labels" tensor is always sorted, i.e. it is of the form [1, 2, 3, ..., 29] (starting at 1, since 0 is background)
            images, targets = batch.values()

            batch_size = images.size(0)
            num_images += batch_size

            images = images.to(device, non_blocking=True)  # shape (batch_size x 1 x 512 x 512)
            targets = [{k: v.to(device, non_blocking=True) for k, v in t.items()} for t in targets]

            # detections is a dict with keys "top_region_boxes" and "top_scores"
            # "top_region_boxes" maps to a tensor of shape [batch_size x 29 x 4]
            # "top_scores" maps to a tensor of shape [batch_size x 29]

            # class_detected is a tensor of shape [batch_size x 29]
            with torch.autocast(device_type='cuda', dtype=torch.float16):
                loss_dict, detections, class_detected = model(images, targets)

            # sum up all 4 losses
            loss = sum(loss for loss in loss_dict.values())
            val_loss += loss.item() * batch_size

            # sum up detections for each class
            sum_class_detected += torch.sum(class_detected, dim=0)

            # compute intersection and union area for each class and add them to the sum
            intersection_area_per_class, union_area_per_class = compute_intersection_and_union_area_per_class(detections, targets, class_detected)
            sum_intersection_area_per_class += intersection_area_per_class
            sum_union_area_per_class += union_area_per_class

            if batch_num == 0:
                plot_gt_and_pred_bboxes_to_tensorboard(writer, overall_steps_taken, images, detections, targets, class_detected, num_images_to_plot=2)

    val_loss /= len(val_dl)
    avg_num_detected_classes_per_image = torch.sum(sum_class_detected / num_images).item()
    avg_detections_per_class = (sum_class_detected / num_images).tolist()
    avg_iou_per_class = (sum_intersection_area_per_class / sum_union_area_per_class).tolist()

    return val_loss, avg_num_detected_classes_per_image, avg_detections_per_class, avg_iou_per_class


def log_stats_to_console(
    train_loss,
    val_loss,
    epoch,
):
    log.info(f"Epoch: {epoch}:")
    log.info(f"\tTrain loss: {train_loss:.3f}")
    log.info(f"\tVal loss: {val_loss:.3f}")



In [11]:

def train_model(
    model,
    train_dl,
    val_dl,
    optimizer,
    scaler,
    lr_scheduler,
    epochs,
    weights_folder_path,
    writer
):
    """
    Train a model on train set and evaluate on validation set.
    Saves best model w.r.t. val loss.

    Parameters
    ----------
    model: nn.Module
        The input model to be trained.
    train_dl: torch.utils.data.Dataloder
        The train dataloader to train on.
    val_dl: torch.utils.data.Dataloder
        The val dataloader to validate on.
    optimizer: Optimizer
        The model's optimizer.
    lr_scheduler: torch.optim.lr_scheduler
        The learning rate scheduler to use.
    epochs: int
        Number of epochs to train for.
    patience: int
        Number of epochs to wait for val loss to decrease.
        If patience is exceeded, then training is stopped early.
    weights_folder_path: str
        Path to folder where best weights will be saved.
    writer: torch.utils.tensorboard.SummaryWriter
        Writer for logging values to tensorboard.

    Returns
    -------
    None, but saves model with the overall lowest val loss at the end of every epoch.
    """
    lowest_val_loss = np.inf

    # the best_model_state is the one where the val loss is the lowest overall
    best_model_state = None

    overall_steps_taken = 0  # for logging to tensorboard

    # for gradient accumulation
    ACCUMULATION_STEPS = EFFECTIVE_BATCH_SIZE // BATCH_SIZE
    
    collected_features = []
#     for epoch in range(epochs):
#         log.info(f"Training epoch {epoch}!")

#         train_loss = 0.0
#         steps_taken = 0
#         missing_count = 0 
    progress_bar = tqdm(enumerate(train_dl), desc="Processing Batches", total=len(train_dl), unit="batch")
    for num_batch, batch in progress_bar:
        # batch is a dict with keys "images" and "targets"
        images, targets = batch.values()

        batch_size = images.size(0)
        images = images.to(device, non_blocking=True)  # shape (batch_size x 1 x 512 x 512)
        targets = [{k: v.to(device, non_blocking=True) for k, v in t.items()} for t in targets]
        with torch.autocast(device_type="cuda", dtype=torch.float16):
            losses, top_region_features, class_detected = model(images, targets)

                # sum up all 4 losses
                #loss = sum(loss for loss in loss_dict.values())

        collected_features.append(top_region_features.detach().cpu().numpy())
    #log.info("Finished training!")
    #log.info(f"Lowest overall val loss: {lowest_val_loss:.3f} at epoch {best_epoch}")
    return collected_features


def collate_fn(batch: List[Dict[str, Tensor]]):
    # each dict in batch (which is a list) is for a single image and has the keys "image", "boxes", "labels"

    # discard images from batch where __getitem__ from custom_image_dataset failed (i.e. returned None)
    # otherwise, whole training loop will stop (even if only 1 image fails to open)
    batch = list(filter(lambda x: x is not None, batch))
    batch = list(filter(lambda x: len(x["boxes"]) > 0 and len(x["labels"]) > 0, batch))# youna add

    image_shape = batch[0]["image"].size()
    # allocate an empty images_batch tensor that will store all images of the batch
    images_batch = torch.empty(size=(len(batch), *image_shape))

    for i, sample in enumerate(batch):
        # remove image tensors from batch and store them in dedicated images_batch tensor
        images_batch[i] = sample.pop("image")

    # since batch (which is a list) now only contains dicts with keys "boxes" and "labels", rename it as targets
    targets = batch

    # create a new batch variable to store images_batch and targets
    batch_new = {}
    batch_new["images"] = images_batch
    batch_new["targets"] = targets

    return batch_new


def get_data_loaders(train_dataset, val_dataset):
    def seed_worker(worker_id):
        """To preserve reproducibility for the randomly shuffled train loader."""
        worker_seed = torch.initial_seed() % 2**32
        np.random.seed(worker_seed)
        random.seed(worker_seed)

    g = torch.Generator()
    g.manual_seed(SEED)

    train_loader = DataLoader(train_dataset, collate_fn=collate_fn, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, 
                              worker_init_fn=seed_worker, generator=g, pin_memory=True)#origin:num_workers=NUM_WORKERS
    val_loader = DataLoader(val_dataset, collate_fn=collate_fn, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)

    return train_loader, val_loader


def get_transforms(dataset: str):
    # see compute_mean_std_dataset.py in src/dataset
    mean = 0.471
    std = 0.302

    # use albumentations for Compose and transforms
    # augmentations are applied with prob=0.5
    # since Affine translates and rotates the image, we also have to do the same with the bounding boxes, hence the bbox_params arugment
    train_transforms = A.Compose(
        [
            # we want the long edge of the image to be resized to IMAGE_INPUT_SIZE, and the short edge of the image to be padded to IMAGE_INPUT_SIZE on both sides,
            # such that the aspect ratio of the images are kept, while getting images of uniform size (IMAGE_INPUT_SIZE x IMAGE_INPUT_SIZE)
            # LongestMaxSize: resizes the longer edge to IMAGE_INPUT_SIZE while maintaining the aspect ratio
            # INTER_AREA works best for shrinking images
            A.LongestMaxSize(max_size=IMAGE_INPUT_SIZE, interpolation=cv2.INTER_AREA),
            A.ColorJitter(hue=0.0),
            A.GaussNoise(),
            # randomly (by default prob=0.5) translate and rotate image
            # mode and cval specify that black pixels are used to fill in newly created pixels
            # translate between -2% and 2% of the image height/width, rotate between -2 and 2 degrees
            A.Affine(mode=cv2.BORDER_CONSTANT, cval=0, translate_percent=(-0.02, 0.02), rotate=(-2, 2)),
            # PadIfNeeded: pads both sides of the shorter edge with 0's (black pixels)
            A.PadIfNeeded(min_height=IMAGE_INPUT_SIZE, min_width=IMAGE_INPUT_SIZE, border_mode=cv2.BORDER_CONSTANT),
            A.Normalize(mean=mean, std=std),
            ToTensorV2()
        ], bbox_params=A.BboxParams(format="pascal_voc", label_fields=['class_labels'])
    )

    # don't apply data augmentations to val and test set
    val_test_transforms = A.Compose(
        [
            A.LongestMaxSize(max_size=IMAGE_INPUT_SIZE, interpolation=cv2.INTER_AREA),
            A.PadIfNeeded(min_height=IMAGE_INPUT_SIZE, min_width=IMAGE_INPUT_SIZE, border_mode=cv2.BORDER_CONSTANT),
            A.Normalize(mean=mean, std=std),
            ToTensorV2(),
        ], bbox_params=A.BboxParams(format="pascal_voc", label_fields=['class_labels'])
    )

    if dataset == "train":
        return train_transforms
    else:
        return val_test_transforms


def get_datasets_as_dfs(config_file_path):
    usecols = ["mimic_image_file_path", "bbox_coordinates", "bbox_labels"]

    # since bbox_coordinates and bbox_labels are stored as strings in the csv_file, we have to apply
    # the literal_eval func to convert them to python lists
    converters = {"bbox_coordinates": literal_eval, "bbox_labels": literal_eval}

    datasets_as_dfs = {dataset: os.path.join(path_full_dataset, dataset) + ".csv" for dataset in ["train", "valid"]}
    datasets_as_dfs = {dataset: pd.read_csv(csv_file_path, usecols=usecols, converters=converters) for dataset, csv_file_path in datasets_as_dfs.items()}
   
    
    for dataset_name, df in datasets_as_dfs.items():
        before_len = len(df)
        
        # bbox_coordinates와 bbox_labels가 유효한 값인지 확인하여 필터링
        df = df.dropna(subset=["bbox_coordinates", "bbox_labels"]).copy()
        def is_valid_target(row):
            return isinstance(row["bbox_coordinates"], list) and len(row["bbox_coordinates"]) > 0 and \
                   isinstance(row["bbox_labels"], list) and len(row["bbox_labels"]) > 0

        df = df[df.apply(is_valid_target, axis=1)].copy()  # 복사하여 안전하게 필터링
        after_len = len(df)
        datasets_as_dfs[dataset_name] = df

        log.info(f"{dataset_name.capitalize()}: Filtered {before_len - after_len} samples without targets.")

    
    total_num_samples_train = len(datasets_as_dfs["train"])
    total_num_samples_val = len(datasets_as_dfs["valid"])

    # compute new number of samples for both train and val
    new_num_samples_train = int(PERCENTAGE_OF_TRAIN_SET_TO_USE * total_num_samples_train)
    new_num_samples_val = int(PERCENTAGE_OF_VAL_SET_TO_USE * total_num_samples_val)

    log.info(f"Train: {new_num_samples_train} images")
    log.info(f"Val: {new_num_samples_val} images")

    with open(config_file_path, "a") as f:
        f.write(f"\tTRAIN NUM IMAGES: {new_num_samples_train}\n")
        f.write(f"\tVAL NUM IMAGES: {new_num_samples_val}\n")
    datasets_as_dfs["train"] = datasets_as_dfs["train"].sample(frac=1, random_state=42).reset_index(drop=True)


    # limit the datasets to those new numbers
    #datasets_as_dfs["train"] = datasets_as_dfs["train"][:new_num_samples_train]
    #datasets_as_dfs["valid"] = datasets_as_dfs["valid"][:new_num_samples_val]
    #print(datasets_as_dfs["train"].head())
    return datasets_as_dfs


def create_run_folder():
    """
    Run folder will contain a folder for saving the trained weights, a folder for the tensorboard files
    as well as a config file that specifies the overall parameters used for training.
    """
    run_folder_path = os.path.join(path_runs_object_detector, f"run_{RUN}")
    weights_folder_path = os.path.join(run_folder_path, "weights")
    tensorboard_folder_path = os.path.join(run_folder_path, "tensorboard")

    # if os.path.exists(run_folder_path): 
    #     log.error(f"Folder to save run {RUN} already exists at {run_folder_path}.")
    #     log.error("Delete the folder or change the run number.")
    #     return None

    os.makedirs(run_folder_path,exist_ok = True)#원래 makedirs
    os.makedirs(weights_folder_path,exist_ok = True)
    os.makedirs(tensorboard_folder_path,exist_ok = True)

    log.info(f"Run {RUN} folder created at {run_folder_path}.")

    config_file_path = os.path.join(run_folder_path, "run_config.txt")
    config_parameters = {
        "COMMENT": RUN_COMMENT,
        "SEED": SEED,
        "IMAGE_INPUT_SIZE": IMAGE_INPUT_SIZE,
        "PERCENTAGE_OF_TRAIN_SET_TO_USE": PERCENTAGE_OF_TRAIN_SET_TO_USE,
        "PERCENTAGE_OF_VAL_SET_TO_USE": PERCENTAGE_OF_VAL_SET_TO_USE,
        "BATCH_SIZE": BATCH_SIZE,
        "EFFECTIVE_BATCH_SIZE": EFFECTIVE_BATCH_SIZE,
        "NUM_WORKERS": NUM_WORKERS,
        "EPOCHS": EPOCHS,
        "LR": LR,
        "EVALUATE_EVERY_K_STEPS": EVALUATE_EVERY_K_STEPS,
        "PATIENCE_LR_SCHEDULER": PATIENCE_LR_SCHEDULER,
        "THRESHOLD_LR_SCHEDULER": THRESHOLD_LR_SCHEDULER,
        "FACTOR_LR_SCHEDULER": FACTOR_LR_SCHEDULER,
        "COOLDOWN_LR_SCHEDULER": COOLDOWN_LR_SCHEDULER
    }

    with open(config_file_path, "w") as f:
        f.write(f"RUN {RUN}:\n")
        for param_name, param_value in config_parameters.items():
            f.write(f"\t{param_name}: {param_value}\n")

    return weights_folder_path, tensorboard_folder_path, config_file_path

In [12]:
def get_datasets(config_file_path):
    usecols = [
        "mimic_image_file_path",
        "bbox_coordinates",
        "bbox_labels",
        "bbox_phrases",
        "bbox_phrase_exists",
        "bbox_is_abnormal",
    ]

    # all of the columns below are stored as strings in the csv_file
    # however, as they are actually lists, we apply the literal_eval func to convert them to lists
    converters = {
        "bbox_coordinates": literal_eval,
        "bbox_labels": literal_eval,
        "bbox_phrases": literal_eval,
        "bbox_phrase_exists": literal_eval,
        "bbox_is_abnormal": literal_eval,
    }

    datasets_as_dfs = {}
    datasets_as_dfs["train"] = pd.read_csv(os.path.join(path_full_dataset, "train.csv"), usecols=usecols, converters=converters)

    # val dataset has additional "reference_report" column
    usecols.append("reference_report")
    datasets_as_dfs["valid"] = pd.read_csv(os.path.join(path_full_dataset, "valid.csv"), usecols=usecols, converters=converters)

    total_num_samples_train = len(datasets_as_dfs["train"])
    total_num_samples_val = len(datasets_as_dfs["valid"])

    # compute new number of samples for both train and val
    new_num_samples_train = int(PERCENTAGE_OF_TRAIN_SET_TO_USE * total_num_samples_train)
    new_num_samples_val = int(PERCENTAGE_OF_VAL_SET_TO_USE * total_num_samples_val)

    log.info(f"Train: {new_num_samples_train} images")
    log.info(f"Val: {new_num_samples_val} images")

    with open(config_file_path, "a") as f:
        f.write(f"\tTRAIN NUM IMAGES: {new_num_samples_train}\n")
        f.write(f"\tVAL NUM IMAGES: {new_num_samples_val}\n")

    # limit the datasets to those new numbers
    datasets_as_dfs["train"] = datasets_as_dfs["train"][:new_num_samples_train]
    datasets_as_dfs["valid"] = datasets_as_dfs["valid"][:new_num_samples_val]

    raw_train_dataset = Dataset.from_pandas(datasets_as_dfs["train"])
    raw_val_dataset = Dataset.from_pandas(datasets_as_dfs["valid"])

    return raw_train_dataset, raw_val_dataset

In [149]:
import ast

# DataFrame의 string으로 저장된 리스트를 실제 리스트로 변환
columns_to_convert = ['bbox_coordinates', 'bbox_labels', 'bbox_phrases', 'bbox_phrase_exists', 'bbox_is_abnormal']

for column in columns_to_convert:
    df_test[column] = df_test[column].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


In [150]:
raw_test_dataset = Dataset.from_pandas(df_test)

In [21]:
weights_folder_path, tensorboard_folder_path, config_file_path = create_run_folder()

datasets_as_dfs = get_datasets_as_dfs(config_file_path) # 여기서 shuffle

[INFO]: Run 14 folder created at D:/CMU/baseline_code/rgrg/runs/object_detector\run_14.


In [20]:
raw_train_dataset, raw_val_dataset = get_datasets(config_file_path)

[INFO]: Train: 128844 images
[INFO]: Val: 6442 images


In [185]:
raw_train_dataset['mimic_image_file_path'][:2]+raw_train_dataset['mimic_image_file_path'][3:9]

['D:/CMU/LSMA/main_project/dataset/mimic_jpg_rgrg\\files/p10/p10000980/s50985099/6ad03ed1-97ee17ee-9cf8b320-f7011003-cd93b42d.jpg',
 'D:/CMU/LSMA/main_project/dataset/mimic_jpg_rgrg\\files/p10/p10000980/s51967283/943486a3-b3fa9ff7-50f5a769-7a62fcbb-f39b6da4.jpg',
 'D:/CMU/LSMA/main_project/dataset/mimic_jpg_rgrg\\files/p10/p10000980/s54935705/6ad819bb-bae74eb9-7b663e90-b8deabd7-57f8054a.jpg',
 'D:/CMU/LSMA/main_project/dataset/mimic_jpg_rgrg\\files/p10/p10000980/s54980801/a75a1fbe-802065ad-717eb7c1-e2ce3552-646276a6.jpg',
 'D:/CMU/LSMA/main_project/dataset/mimic_jpg_rgrg\\files/p10/p10000980/s57861150/5aa15ba6-55f5e96e-39cea686-7c3b28b2-b8c97a88.jpg',
 'D:/CMU/LSMA/main_project/dataset/mimic_jpg_rgrg\\files/p10/p10000980/s58206436/54affd39-8bf24209-232bac8a-df6c277a-398ee8a5.jpg',
 'D:/CMU/LSMA/main_project/dataset/mimic_jpg_rgrg\\files/p10/p10000980/s58636672/c7526473-7b7214ee-a5d58d12-29d1f67f-9f4edf00.jpg',
 'D:/CMU/LSMA/main_project/dataset/mimic_jpg_rgrg\\files/p10/p10000980/s5998

예외 처리 : 'D:/CMU/LSMA/main_project/dataset/mimic_jpg_rgrg\\files/p10/p10000980/s54577367/cfb03587-782edf6c-1bf392e1-98196cd5-365d69e8.jpg', 0,1,2번째임. 

In [ ]:
# 0,1,2 번째 

'D:/CMU/LSMA/main_project/dataset/mimic_jpg_rgrg\\files/p10/p10000980/s54577367/cfb03587-782edf6c-1bf392e1-98196cd5-365d69e8.jpg',

In [184]:
filtered_df['mimic_image_file_path'].iloc[2]# 하 세번째 꺼임.. 없는거만 예외 처리 할까?

'D:/CMU/LSMA/main_project/dataset/mimic_jpg_rgrg\\files/p10/p10000980/s54935705/6ad819bb-bae74eb9-7b663e90-b8deabd7-57f8054a.jpg'

In [ ]:
del dataset_as_dfs

In [21]:
# 조건에 해당하는 파일 경로 리스트
target_files = raw_train_dataset['mimic_image_file_path'][:2]+raw_train_dataset['mimic_image_file_path'][3:9]

# DataFrame에서 조건에 맞는 행 추출
filtered_df = datasets_as_dfs['train'][datasets_as_dfs['train']['mimic_image_file_path'].isin(target_files)].copy()

# 순서 보장 (target_files 순서대로)
filtered_df['mimic_image_file_path'] = pd.Categorical(filtered_df['mimic_image_file_path'], categories=target_files, ordered=True)
filtered_df = filtered_df.sort_values('mimic_image_file_path')

# 결과 출력
print(filtered_df.head())


                                    mimic_image_file_path  \
62943   D:/CMU/LSMA/main_project/dataset/mimic_jpg_rgr...   
42768   D:/CMU/LSMA/main_project/dataset/mimic_jpg_rgr...   
101021  D:/CMU/LSMA/main_project/dataset/mimic_jpg_rgr...   
13727   D:/CMU/LSMA/main_project/dataset/mimic_jpg_rgr...   
48705   D:/CMU/LSMA/main_project/dataset/mimic_jpg_rgr...   

                                         bbox_coordinates       bbox_labels  
62943   [[225, 144, 512, 553], [406, 171, 512, 550], [...  [1, 2, 3, 6, 19]  
42768   [[231, 109, 512, 615], [504, 163, 512, 615], [...        [1, 2, 19]  
101021  [[313, 300, 512, 615], [450, 341, 512, 615], [...     [1, 2, 6, 19]  
13727        [[231, 532, 512, 615], [477, 573, 512, 615]]            [1, 2]  
48705                              [[450, 532, 512, 615]]               [1]  


In [89]:
filtered_df.to_csv('binary_df.csv', index=False)

In [179]:
raw_train_dataset['mimic_image_file_path'][:9]

['D:/CMU/LSMA/main_project/dataset/mimic_jpg_rgrg\\files/p10/p10000980/s50985099/6ad03ed1-97ee17ee-9cf8b320-f7011003-cd93b42d.jpg',
 'D:/CMU/LSMA/main_project/dataset/mimic_jpg_rgrg\\files/p10/p10000980/s51967283/943486a3-b3fa9ff7-50f5a769-7a62fcbb-f39b6da4.jpg',
 'D:/CMU/LSMA/main_project/dataset/mimic_jpg_rgrg\\files/p10/p10000980/s54577367/cfb03587-782edf6c-1bf392e1-98196cd5-365d69e8.jpg',
 'D:/CMU/LSMA/main_project/dataset/mimic_jpg_rgrg\\files/p10/p10000980/s54935705/6ad819bb-bae74eb9-7b663e90-b8deabd7-57f8054a.jpg',
 'D:/CMU/LSMA/main_project/dataset/mimic_jpg_rgrg\\files/p10/p10000980/s54980801/a75a1fbe-802065ad-717eb7c1-e2ce3552-646276a6.jpg',
 'D:/CMU/LSMA/main_project/dataset/mimic_jpg_rgrg\\files/p10/p10000980/s57861150/5aa15ba6-55f5e96e-39cea686-7c3b28b2-b8c97a88.jpg',
 'D:/CMU/LSMA/main_project/dataset/mimic_jpg_rgrg\\files/p10/p10000980/s58206436/54affd39-8bf24209-232bac8a-df6c277a-398ee8a5.jpg',
 'D:/CMU/LSMA/main_project/dataset/mimic_jpg_rgrg\\files/p10/p10000980/s5863

In [91]:
# Dataset 데이터를 Dictionary로 변환
dataset_dict = {
    'mimic_image_file_path': raw_train_dataset['mimic_image_file_path'],
    'bbox_coordinates': raw_train_dataset['bbox_coordinates'],
    'bbox_labels': raw_train_dataset['bbox_labels'],
    'bbox_phrases': raw_train_dataset['bbox_phrases'],
    'bbox_phrase_exists': raw_train_dataset['bbox_phrase_exists'],
    'bbox_is_abnormal': raw_train_dataset['bbox_is_abnormal'],
}

# 확인
print(dataset_dict.keys())  # Dictionary의 키 확인
import pickle

# Pickle 파일로 저장
with open("raw_train_dict.pkl", "wb") as f:
    pickle.dump(dataset_dict, f)

# Pickle 파일 로드
# with open("dataset_dict.pkl", "rb") as f:
#     loaded_dict = pickle.load(f)

# print(loaded_dict['mimic_image_file_path'][:5])  # 일부 데이터 확인

dict_keys(['mimic_image_file_path', 'bbox_coordinates', 'bbox_labels', 'bbox_phrases', 'bbox_phrase_exists', 'bbox_is_abnormal'])


In [1]:
import pandas as pd

In [5]:
del df_all

In [3]:
df_all = pd.read_csv(r"D:\CMU\LSMA\main_project\full_dataset\train.csv")

In [90]:
from datasets import Dataset

In [16]:
def validate_datasets(datasets_as_dfs):
    for dataset_name, df in datasets_as_dfs.items():
        # 1. bbox_coordinates와 bbox_labels가 모두 list인지 확인
        invalid_format = df[
            ~df["bbox_coordinates"].apply(lambda x: isinstance(x, list)) |
            ~df["bbox_labels"].apply(lambda x: isinstance(x, list))
        ]
        if not invalid_format.empty:
            print(f"[{dataset_name}] Invalid format rows found:")
            print(invalid_format)

        # 2. bbox_coordinates와 bbox_labels의 길이가 같은지 확인
        invalid_lengths = df[
            df.apply(lambda row: len(row["bbox_coordinates"]) != len(row["bbox_labels"]), axis=1)
        ]
        if not invalid_lengths.empty:
            print(f"[{dataset_name}] Mismatched lengths between bbox_coordinates and bbox_labels:")
            print(invalid_lengths)

    print("Validation complete.")
validate_datasets(datasets_as_dfs)

Validation complete.


In [ ]:
dataset_as_dfs['train']을 주작하자. 

In [32]:
train_transforms = get_transforms("train")
val_transforms = get_transforms("val")

train_dataset = CustomImageDataset(datasets_as_dfs["train"], train_transforms)
val_dataset = CustomImageDataset(datasets_as_dfs["valid"], val_transforms)

train_loader, val_loader = get_data_loaders(train_dataset, val_dataset)

model = ObjectDetector(return_feature_vectors=True)

model.to(device, non_blocking=True)
model.train()

scaler = torch.amp.GradScaler('cuda')

opt = AdamW(model.parameters(), lr=LR)
lr_scheduler = ReduceLROnPlateau(opt, mode="min", factor=FACTOR_LR_SCHEDULER, patience=PATIENCE_LR_SCHEDULER, threshold=THRESHOLD_LR_SCHEDULER, cooldown=COOLDOWN_LR_SCHEDULER)
writer = SummaryWriter(log_dir=tensorboard_folder_path)
log.info("\nStarting training!\n")

collected_features_train = train_model(
    model=model,
    train_dl=train_loader,
    val_dl=val_loader,
    optimizer=opt,
    scaler=scaler,
    lr_scheduler=lr_scheduler,
    epochs=EPOCHS,
    weights_folder_path=weights_folder_path,
    writer=writer
)



[INFO]: 
Starting training!

Processing Batches: 100%|███████████████████████████████████████| 7015/7015 [6:49:15<00:00,  3.50s/batch]


필터링 된거 뽑는거임

In [17]:
test_df = pd.read_csv(r"D:\CMU\LSMA\main_project\full_dataset\test-2.csv")
df_test = test_df[test_df['image_id']=='dcaead7c-bba30dd7-76fd5755-24ac78ea-2222e886'].copy()

In [62]:
import cv2
import torch
from torch.utils.data import Dataset


class CustomImageDataset(Dataset):
    def __init__(self, dataset_df, transforms):
        super().__init__()
        self.dataset_df = dataset_df
        self.transforms = transforms

    def __len__(self):
        return len(self.dataset_df)

    def __getitem__(self, index):
        # if something in __get__item fails, then return None
        # collate_fn in dataloader filters out None values
        try:
            # mimic_image_file_path is the 1st column of the dataframes
            image_path = self.dataset_df['mimic_image_file_path'].iloc[index]

            # cv2.imread by default loads an image with 3 channels
            # since we have grayscale images, we only have 1 channel and thus use cv2.IMREAD_UNCHANGED to read in the 1 channel
            image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)

            # bbox_coordinates (List[List[int]]) is the 2nd column of the dataframes
            bbox_coordinates = self.dataset_df['bbox_coordinates'].iloc[index]

            # bbox_labels (List[int]) is the 3rd column of the dataframes
            class_labels = self.dataset_df['bbox_labels'].iloc[index]
            if isinstance(bbox_coordinates, str):
                bbox_coordinates = ast.literal_eval(bbox_coordinates)

            if isinstance(class_labels, str):
                class_labels = ast.literal_eval(class_labels)
            # apply transformations to image, bboxes and label
            transformed = self.transforms(image=image, bboxes=bbox_coordinates, class_labels=class_labels)

            transformed_image = transformed["image"]
            transformed_bboxes = transformed["bboxes"]
            transformed_bbox_labels = transformed["class_labels"]

            sample = {
                "image": transformed_image,
                "boxes": torch.tensor(transformed_bboxes, dtype=torch.float),
                "labels": torch.tensor(transformed_bbox_labels, dtype=torch.int64),
            }
        except Exception:
            print(f"Error processing index {index}: {e}")
            return None

        return sample


In [63]:
test_dataset = CustomImageDataset(df_test, train_transforms)


test_loader, test1_loader = get_data_loaders(test_dataset, test_dataset)

model = ObjectDetector(return_feature_vectors=True)

model.to(device, non_blocking=True)


scaler = torch.amp.GradScaler('cuda')

opt = AdamW(model.parameters(), lr=LR)
lr_scheduler = ReduceLROnPlateau(opt, mode="min", factor=FACTOR_LR_SCHEDULER, patience=PATIENCE_LR_SCHEDULER, threshold=THRESHOLD_LR_SCHEDULER, cooldown=COOLDOWN_LR_SCHEDULER)
writer = SummaryWriter(log_dir=tensorboard_folder_path)
log.info("\nStarting training!\n")

[INFO]: 
Starting training!



In [64]:
object_detector_checkpoint = {
    key.replace("object_detector.", ""): value
    for key, value in checkpoint['model'].items()
    if "binary" not in key and "language" not in key
}

In [65]:
object_detector_state_dict = model.state_dict()

In [66]:
for key, value in object_detector_checkpoint.items():
    if key in object_detector_state_dict:
        object_detector_state_dict[key].copy_(value)
    else:
        print(f"Skipping key not in model: {key}")

# 업데이트된 state_dict 로드
model.load_state_dict(object_detector_state_dict, strict=False)
model.eval()

ObjectDetector(
  (backbone): Sequential(
    (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Con

In [61]:
df_test['bbox_coordinates'].iloc[0]

'[[144, 315, 512, 512], [315, 360, 512, 512], [433, 315, 512, 512], [180, 144, 512, 512]]'

In [68]:
import ast

In [70]:

lowest_val_loss = np.inf

# the best_model_state is the one where the val loss is the lowest overall
best_model_state = None

overall_steps_taken = 0  # for logging to tensorboard

# for gradient accumulation
ACCUMULATION_STEPS = EFFECTIVE_BATCH_SIZE // BATCH_SIZE

object_test_f = []
#     for epoch in range(epochs):
#         log.info(f"Training epoch {epoch}!")

#         train_loss = 0.0
#         steps_taken = 0
#         missing_count = 0 

progress_bar = tqdm(enumerate(test_loader), desc="Processing Batches", total=len(test_loader), unit="batch")
for num_batch, batch in progress_bar:
    # batch is a dict with keys "images" and "targets"
    images, targets = batch.values()

    batch_size = images.size(0)
    images = images.to(device, non_blocking=True)  # shape (batch_size x 1 x 512 x 512)
    targets = [{k: v.to(device, non_blocking=True) for k, v in t.items()} for t in targets]
    with torch.autocast(device_type="cuda", dtype=torch.float16):
        losses,d, top_region_features, class_detected = model(images, targets)

            # sum up all 4 losses
            #loss = sum(loss for loss in loss_dict.values())

    object_test_f.append(top_region_features.detach().cpu().numpy())



Processing Batches: 100%|███████████████████████████████████████████████| 1/1 [00:00<00:00,  5.84batch/s]


In [72]:
top_region_features.shape

torch.Size([1, 29, 1024])

In [74]:
import pickle

In [75]:
top_region_features_list = top_region_features.tolist()
# 저장할 파일 경로
save_path = "object_feature_test.pkl"

# Pickle로 저장
with open(save_path, "wb") as f:
    pickle.dump(top_region_features_list, f)

print(f"Top region features saved to {save_path}")


Top region features saved to object_feature_test.pkl


In [29]:
collected_features3 = train_model(
    model=model,
    train_dl=test_loader,
    val_dl=test_loader,
    optimizer=opt,
    scaler=scaler,
    lr_scheduler=lr_scheduler,
    epochs=EPOCHS,
    weights_folder_path=weights_folder_path,
    writer=writer
)



Processing Batches:   0%|                                                       | 0/1 [00:00<?, ?batch/s]


NameError: name 'e' is not defined

In [37]:
col_feature_t = collected_features2

In [36]:
len(collected_features_train[0])

16

In [40]:
def get_datasets_as_dfs_2(config_file_path):
    usecols = ["mimic_image_file_path", "bbox_coordinates", "bbox_labels"]

    # since bbox_coordinates and bbox_labels are stored as strings in the csv_file, we have to apply
    # the literal_eval func to convert them to python lists
    converters = {"bbox_coordinates": literal_eval, "bbox_labels": literal_eval}

    datasets_as_dfs = {dataset: os.path.join(path_full_dataset, dataset) + ".csv" for dataset in ["train", "valid"]}
    datasets_as_dfs = {dataset: pd.read_csv(csv_file_path, converters=converters) for dataset, csv_file_path in datasets_as_dfs.items()}
   
    
    for dataset_name, df in datasets_as_dfs.items():
        before_len = len(df)
        
        # bbox_coordinates와 bbox_labels가 유효한 값인지 확인하여 필터링
        df = df.dropna(subset=["bbox_coordinates", "bbox_labels"]).copy()
        def is_valid_target(row):
            return isinstance(row["bbox_coordinates"], list) and len(row["bbox_coordinates"]) > 0 and \
                   isinstance(row["bbox_labels"], list) and len(row["bbox_labels"]) > 0

        df = df[df.apply(is_valid_target, axis=1)].copy()  # 복사하여 안전하게 필터링
        after_len = len(df)
        datasets_as_dfs[dataset_name] = df

        log.info(f"{dataset_name.capitalize()}: Filtered {before_len - after_len} samples without targets.")

    
    total_num_samples_train = len(datasets_as_dfs["train"])
    total_num_samples_val = len(datasets_as_dfs["valid"])

    # compute new number of samples for both train and val
    new_num_samples_train = int(PERCENTAGE_OF_TRAIN_SET_TO_USE * total_num_samples_train)
    new_num_samples_val = int(PERCENTAGE_OF_VAL_SET_TO_USE * total_num_samples_val)

    log.info(f"Train: {new_num_samples_train} images")
    log.info(f"Val: {new_num_samples_val} images")

    with open(config_file_path, "a") as f:
        f.write(f"\tTRAIN NUM IMAGES: {new_num_samples_train}\n")
        f.write(f"\tVAL NUM IMAGES: {new_num_samples_val}\n")
    datasets_as_dfs["train"] = datasets_as_dfs["train"].sample(frac=1, random_state=42).reset_index(drop=True)


    # limit the datasets to those new numbers
    #datasets_as_dfs["train"] = datasets_as_dfs["train"][:new_num_samples_train]
    #datasets_as_dfs["valid"] = datasets_as_dfs["valid"][:new_num_samples_val]
    #print(datasets_as_dfs["train"].head())
    return datasets_as_dfs


In [152]:
datasets_as_dfs["train"].head()

,mimic_image_file_path,bbox_coordinates,bbox_labels
0,D:/CMU/LSMA/main_project/dataset/mimic_jpg_rgr...,"[[341, 40, 512, 426], [177, 177, 512, 426]]","[1, 19]"
1,D:/CMU/LSMA/main_project/dataset/mimic_jpg_rgr...,"[[81, 532, 512, 615]]",[19]
2,D:/CMU/LSMA/main_project/dataset/mimic_jpg_rgr...,"[[95, 504, 512, 615], [313, 559, 512, 615], [4...","[1, 2, 6, 19]"
3,D:/CMU/LSMA/main_project/dataset/mimic_jpg_rgr...,"[[18, 433, 512, 596], [252, 469, 512, 596], [3...","[1, 2, 6, 19]"
4,D:/CMU/LSMA/main_project/dataset/mimic_jpg_rgr...,"[[497, 189, 512, 489]]",[1]


In [152]:
filtered_df.head(1)

,mimic_image_file_path,bbox_coordinates,bbox_labels
62943,D:/CMU/LSMA/main_project/dataset/mimic_jpg_rgr...,"[[225, 144, 512, 553], [406, 171, 512, 550], [...","[1, 2, 3, 6, 19]"


In [42]:
# 피처 프린트 해보고, len확인해보기, feature 확인
print(len(datasets_as_dfs["train"]))
print(len(collected_features_train))

112240
7015


In [109]:
def collate_fn_f(batch: List[Dict[str, Tensor]], batch_index: int = 0):
    # 필터링 기록을 저장할 리스트
    filtered_indices = []

    # 필터링 전 원래 배치 크기
    original_batch_size = len(batch)
    print(type(batch))
    # __getitem__ 실패한 데이터 필터링
    batch = list(filter(lambda x: x is not None, batch))
    if len(batch) < original_batch_size:
        filtered_indices.append((batch_index, "getitem_failed"))

    # boxes와 labels가 없는 데이터 필터링
    before_second_filter_size = len(batch)
    batch = list(filter(lambda x: len(x["boxes"]) > 0 and len(x["labels"]) > 0, batch))
    if len(batch) < before_second_filter_size:
        filtered_indices.append((batch_index, "empty_boxes_or_labels"))

    image_shape = batch[0]["image"].size()
    images_batch = torch.empty(size=(len(batch), *image_shape))

    for i, sample in enumerate(batch):
        images_batch[i] = sample.pop("image")

    targets = batch

    batch_new = {}
    batch_new["images"] = images_batch
    batch_new["targets"] = targets

    # 필터링된 데이터 기록 반환 (디버깅용)
    return batch_new, filtered_indices


In [68]:
def get_data_loaders_2(train_dataset):
    def seed_worker(worker_id):
        """To preserve reproducibility for the randomly shuffled train loader."""
        worker_seed = torch.initial_seed() % 2**32
        np.random.seed(worker_seed)
        random.seed(worker_seed)

    g = torch.Generator()
    g.manual_seed(SEED)

    train_loader = DataLoader(train_dataset, collate_fn=collate_fn_f, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, 
                              worker_init_fn=seed_worker, generator=g, pin_memory=True)#origin:num_workers=NUM_WORKERS
    #val_loader = DataLoader(val_dataset, collate_fn=collate_fn, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)

    return train_loader

In [69]:
train_loader2 = get_data_loaders_2(train_dataset)

In [132]:
# 데이터로더 루프에서 추적
filtered_log = []

for batch_index, batch in enumerate(train_loader2):
    batch = batch[0]['targets']
    #print(batch)
    batch = list(filter(lambda x: x is not None, batch))
    batch = list(filter(lambda x: len(x["boxes"]) > 0 and len(x["labels"]) > 0, batch))# youna add
    if len(batch)!=16:
        print(batch_index,'는',len(batch))
    #batch_new, filtered_indices = collate_fn_f(batch[0], batch_index=batch_index)
    #if filtered_indices:
    #    filtered_log.extend(filtered_indices)
    #break
# 필터링된 데이터 출력
#print("Filtered data log:", filtered_log)


59 는 15
83 는 15
102 는 15
169 는 15
196 는 15
252 는 15
326 는 15
331 는 15
332 는 15
354 는 15
452 는 15
487 는 15
499 는 15
518 는 15
574 는 15
639 는 15
742 는 15
831 는 15
945 는 15
1122 는 15
1128 는 15
1130 는 15
1136 는 15
1204 는 15
1235 는 15
1243 는 15
1302 는 15
1351 는 14
1429 는 15
1486 는 15
1527 는 15
1547 는 15
1574 는 15
1578 는 15
1758 는 15
1779 는 15
1798 는 15
1881 는 15
1884 는 15
1938 는 15
1945 는 15
2007 는 15
2080 는 15
2127 는 15
2212 는 15
2218 는 15
2232 는 15
2251 는 15
2266 는 15
2325 는 15
2331 는 15
2355 는 15
2364 는 15
2375 는 15
2391 는 15
2429 는 15
2521 는 15
2611 는 15
2762 는 15
2771 는 15
2778 는 15
2794 는 15
2862 는 15
3003 는 15
3006 는 15
3027 는 15
3035 는 15
3049 는 15
3076 는 15
3098 는 15
3216 는 15
3274 는 15
3430 는 15
3433 는 15
3450 는 15
3453 는 15
3460 는 15
3461 는 15
3536 는 15
3538 는 15
3643 는 15
3735 는 15
3757 는 15
3762 는 15
3817 는 15
3904 는 15
3978 는 15
4028 는 15
4099 는 15
4147 는 15
4250 는 15
4292 는 15
4316 는 15
4357 는 15
4410 는 15
4481 는 15
4488 는 15
4653 는 15
4663 는 15
4741 는 15
4742 는 15
4750 는 15
4

In [ ]:
# dataset_as_dfs2랑 매칭하기

In [52]:
datasets_as_dfs2 = get_datasets_as_dfs_2(config_file_path)

[INFO]: Train: Filtered 16604 samples without targets.
[INFO]: Valid: Filtered 4029 samples without targets.
[INFO]: Train: 112240 images
[INFO]: Val: 5636 images


In [54]:
len(datasets_as_dfs2['train'])

112240

In [107]:
112240/16

7015.0

In [108]:
(112240-112079)

161

In [190]:
len(collected_features_train)

7015

In [218]:
# 데이터 저장
file_path = "collected_features_train_zip.npy"  # 저장할 파일 경로
np.save(file_path, collected_features_train)

C:\Users\seclab\anaconda3\envs\rgrg_env\lib\site-packages\numpy\lib\npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [50]:
collected_feature_train_all = []
for batch in tqdm(collected_features_train, desc="Combining batches", unit="batch"):
    collected_feature_train_all.extend(batch)
print(len(collected_feature_train_all))  
# Step 3: 최종 결과 저장 (예: NumPy 파일로 저장)
np.save("collected_feature_train_all.npy", np.array(collected_feature_train_all))

Combining batches: 100%|██████████████████████████████████████| 7015/7015 [00:00<00:00, 342937.89batch/s]


112079


In [44]:
import torch

# Checkpoint 파일 로드
checkpoint_path = r"D:\CMU\baseline_code\full_model_checkpoint_val_loss_19.793_overall_steps_155252.pt"  # 실제 파일 경로
checkpoint = torch.load(checkpoint_path)

# 파일 구조 확인
print("Checkpoint keys:", checkpoint.keys())  # 포함된 키 확인

# 모델의 state_dict 확인
if "state_dict" in checkpoint:
    print("State_dict keys:", checkpoint["state_dict"].keys())

# 기타 정보 확인
if "optimizer" in checkpoint:
    print("Optimizer keys:", checkpoint["optimizer"].keys())


C:\Users\seclab\AppData\Local\Temp\ipykernel_37240\1779270865.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)


Checkpoint keys: dict_keys(['model', 'optimizer', 'scaler', 'current_epoch', 'overall_steps_taken', 'lowest_val_loss'])
Optimizer keys: dict_keys(['state', 'param_groups'])


## binary model

In [76]:
cp_binary = []

for key in checkpoint['model'].keys():
    a = 'binary_classifier_region_selection'
    if a in key:
        cp_binary.append(key)

In [138]:
cp_binary

['binary_classifier_region_selection.classifier.0.weight',
 'binary_classifier_region_selection.classifier.0.bias',
 'binary_classifier_region_selection.classifier.2.weight',
 'binary_classifier_region_selection.classifier.2.bias',
 'binary_classifier_region_selection.classifier.4.weight',
 'binary_classifier_region_selection.classifier.4.bias',
 'binary_classifier_region_selection.loss_fn.pos_weight']

In [134]:
print(checkpoint['optimizer'].keys())
print(checkpoint['scaler'].keys())
print(checkpoint['current_epoch'])


dict_keys(['state', 'param_groups'])
dict_keys(['scale', 'growth_factor', 'backoff_factor', 'growth_interval', '_growth_tracker'])
1


In [144]:
print(f"W : {checkpoint['model']['binary_classifier_region_selection.classifier.0.weight'].shape}, b : {checkpoint['model']['binary_classifier_region_selection.classifier.0.bias'].shape}")

W : torch.Size([512, 1024]), b : torch.Size([512])


In [148]:
print(f"W : {checkpoint['model']['binary_classifier_region_selection.classifier.0.weight']}")

W : tensor([[-0.0358, -0.0185,  0.0579,  ...,  0.0232, -0.0233,  0.0224],
        [ 0.0400,  0.0119,  0.0498,  ...,  0.1066, -0.0391,  0.0423],
        [ 0.0158,  0.0090,  0.0265,  ...,  0.0820, -0.0228,  0.0541],
        ...,
        [-0.0030, -0.0132,  0.0567,  ...,  0.0418, -0.0516,  0.0076],
        [ 0.0053, -0.0460,  0.0769,  ..., -0.0593, -0.0729, -0.0254],
        [ 0.0750,  0.0245,  0.1156,  ..., -0.0080, -0.0075,  0.0215]],
       device='cuda:0')


In [63]:
import torch
import torch.nn as nn

class BinaryClassifierRegionSelection(nn.Module):
    def __init__(self):
        super().__init__()

        self.classifier = nn.Sequential(
            nn.Linear(in_features=1024, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=1)
        )

        # since we have around 2.2x more regions without sentences than regions with sentences (see dataset/dataset_stats.txt generated from compute_stats_dataset.py),
        # we set pos_weight=2.2 to put 2.2 more weight on the loss of regions with sentences
        pos_weight = torch.tensor([2.2], device=device)
        self.loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    def forward(
        self,
        top_region_features,  # tensor of shape [batch_size x 29 x 1024]
        class_detected,  # boolean tensor of shape [batch_size x 29], indicates if the object detector has detected the region/class or not
        return_loss,  # boolean value that is True if we need the loss (necessary for training and evaluation)
        region_has_sentence=None  # boolean tensor of shape [batch_size x 29], indicates if a region has a sentence (True) or not (False) as the ground truth
    ):
        # logits of shape [batch_size x 29]
        logits = self.classifier(top_region_features).squeeze(dim=-1)

        # the loss is needed for training and evaluation
        if return_loss:
            # only compute loss for logits that correspond to a class that was detected
            detected_logits = logits[class_detected]
            detected_region_has_sentence = region_has_sentence[class_detected]

            loss = self.loss_fn(detected_logits, detected_region_has_sentence.type(torch.float32))

        if self.training:
            return loss
        else:
            # compute selected_regions for evaluation and inference
            # selected_regions are the predictions by the classifier, has shape [batch_size x 29] and is True for regions that should get a sentence
            # selected_regions are needed to filter the top_region_features to those that the classifier selected for sentence generation,
            # to evaluate the classifier during eval mode (comparing selected_regions to ground-truth region_has_sentence to compute recall, precision etc.)
            # and to generally know which regions were selected for generation (which is needed for mapping each selected_region_feature to its corresponding class)
            #
            # use a threshold of -1 in logit-space (i.e. 0.269 in probability-space)
            # if a logit > -1, then it means that class/region has boolean value True and a sentence should be generated for it
            selected_regions = logits > -1

            # set to False all regions that were not detected by object detector
            # (since no detection -> no sentence generation possible)
            selected_regions[~class_detected] = False

            # selected_region_features are inputted into the decoder during evaluation and inference to generate the sentences
            # selected_region_features is of shape [num_regions_selected_in_batch, 1024]
            selected_region_features = top_region_features[selected_regions]

            # if in eval mode
            if return_loss:
                return loss, selected_regions, selected_region_features
            else:
                # if in inference mode
                return selected_regions, selected_region_features


In [64]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# 모델 초기화
model = BinaryClassifierRegionSelection().to(device)


model.load_state_dict({
    'classifier.0.weight': checkpoint['model']['binary_classifier_region_selection.classifier.0.weight'],
    'classifier.0.bias': checkpoint['model']['binary_classifier_region_selection.classifier.0.bias'],
    'classifier.2.weight': checkpoint['model']['binary_classifier_region_selection.classifier.2.weight'],
    'classifier.2.bias': checkpoint['model']['binary_classifier_region_selection.classifier.2.bias'],
    'classifier.4.weight': checkpoint['model']['binary_classifier_region_selection.classifier.4.weight'],
    'classifier.4.bias': checkpoint['model']['binary_classifier_region_selection.classifier.4.bias'],
    'loss_fn.pos_weight':checkpoint['model']['binary_classifier_region_selection.loss_fn.pos_weight']
})


#model.loss_fn.pos_weight = checkpoint['model']['binary_classifier_region_selection.loss_fn.pos_weight']
# 학습 루프
model.eval()



batch_size = 8  # 예시 배치 크기



In [182]:
target_s = []

for i in datasets_as_dfs2['train']['bbox_phrase_exists'].iloc[:16]:
    target_s.append(eval(i))

In [183]:
len(target_s)

16

In [189]:
len(collected_feature_train_all)

112079

In [203]:
# 4번째 샘플 제외하고 17번째까지 선택
filtered_features = [feature for i, feature in enumerate(collected_features_train[:17]) if i != 4]

# 선택한 샘플 출력
for i, feature in enumerate(filtered_features):
    print(f"Sample {i}: {feature.shape}")


Sample 0: (16, 29, 1024)
Sample 1: (16, 29, 1024)
Sample 2: (16, 29, 1024)
Sample 3: (16, 29, 1024)
Sample 4: (16, 29, 1024)
Sample 5: (16, 29, 1024)
Sample 6: (16, 29, 1024)
Sample 7: (16, 29, 1024)
Sample 8: (16, 29, 1024)
Sample 9: (16, 29, 1024)
Sample 10: (16, 29, 1024)
Sample 11: (16, 29, 1024)
Sample 12: (16, 29, 1024)
Sample 13: (16, 29, 1024)
Sample 14: (16, 29, 1024)
Sample 15: (16, 29, 1024)


In [205]:
collected_features_train_.shape

torch.Size([16, 16, 29, 1024])

In [206]:
target_s_.shape

torch.Size([16, 29])

In [25]:
# 데이터 변환
file_path = 'collected_features_train_zip.npy'
collected_features_train = np.load(file_path, allow_pickle=True)
# npy 파일 로드




In [77]:
df_test.head(1)

,subject_id,study_id,image_id,mimic_image_file_path,bbox_coordinates,bbox_labels,bbox_phrases,bbox_phrase_exists,bbox_is_abnormal,reference_report
89,10019777,56013693,dcaead7c-bba30dd7-76fd5755-24ac78ea-2222e886,D:/CMU/LSMA/main_project/dataset/mimic_jpg_rgr...,"[[144, 315, 512, 512], [315, 360, 512, 512], [...","[1, 2, 6, 19]","[""Lung volumes are low resulting crowding of t...","[True, False, True, True, True, False, True, F...","[True, False, True, True, True, False, False, ...",Lung volumes are low resulting crowding of the...


In [67]:
target_s = raw_train_dataset['bbox_phrase_exists'][:8]

In [71]:

collected_features_train_ = torch.tensor(col_feature_t[0], dtype=torch.float32).to(device)

In [72]:

target_s_ = torch.tensor(target_s, dtype=torch.bool).to(device)

# 추론
with torch.no_grad():  # 추론 중에는 기울기 계산 비활성화
    selected_regions, selected_region_features = model(
        top_region_features=collected_features_train_,
        class_detected=target_s_,
        return_loss=False  # 추론 모드
    )

print("Selected Regions:", selected_regions)
print("Selected Region Features:", selected_region_features)


Selected Regions: tensor([[ True, False, False, False, False, False,  True, False, False, False,
         False, False,  True, False,  True, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False],
        [ True,  True,  True, False,  True,  True,  True, False,  True, False,
         False, False, False, False,  True, False, False, False, False, False,
         False, False, False, False,  True, False, False, False, False],
        [ True,  True, False, False,  True, False,  True, False,  True,  True,
         False, False,  True, False,  True, False, False, False, False, False,
          True,  True, False, False,  True, False, False, False, False],
        [ True,  True, False, False,  True, False,  True, False,  True,  True,
         False, False,  True, False,  True, False, False,  True,  True,  True,
          True,  True, False, False,  True, False, False, False, False],
        [ True, False, False, False,  True, False,  True, 

SHAP

In [81]:
print(collected_features_train_.shape)
print(target_s_.shape)

torch.Size([8, 29, 1024])
torch.Size([8, 29])


In [92]:
target_s_

tensor([[ True, False, False, False,  True, False,  True, False,  True, False,
         False, False,  True, False,  True, False, False, False, False, False,
         False,  True, False, False,  True, False, False, False, False],
        [ True,  True,  True, False,  True,  True,  True, False,  True, False,
         False, False, False, False,  True, False, False, False, False, False,
         False, False, False, False,  True, False, False, False, False],
        [ True,  True, False, False,  True, False,  True, False,  True,  True,
         False, False,  True, False,  True, False, False, False, False, False,
          True,  True, False, False,  True, False, False, False, False],
        [ True,  True, False, False,  True, False,  True, False,  True,  True,
         False, False,  True, False,  True, False, False,  True,  True,  True,
          True,  True, False, False,  True, False, False, False, False],
        [ True, False, False, False,  True, False,  True, False,  True, Fals

In [93]:
# 텐서를 NumPy 배열로 변환 후 저장
np.save("target_s_array.npy", target_s_.cpu().numpy())
np.save("collected_features_train_array.npy", collected_features_train_.cpu().numpy())

In [ ]:
# NumPy 배열 로드 후 다시 텐서로 변환
target_s_ = np.load("target_s_array.npy")
target_s_ = torch.tensor(target_s_)
collected_features_train_ = np.load("collected_features_train_array.npy")
collected_features_train_ = torch.tensor(collected_features_train_)

In [96]:
import shap

model.eval()  # 평가 모드

# Input 데이터 변환: [batch_size, 29, 1024] -> [batch_size, 29 * 1024]
X = collected_features_train_.reshape(collected_features_train_.shape[0], -1).cpu().numpy()

def shap_model_fn(x):
    # SHAP에서 입력받은 x는 [1, 29 * 1024] 형태
    x_tensor = torch.tensor(x, dtype=torch.float32).to(next(model.parameters()).device)
    x_tensor = x_tensor.view(x_tensor.shape[0], 29, 1024)  # 원래 모델 입력 형태로 변환

    # `class_detected`의 shape을 SHAP에서 전달된 batch size에 맞게 변경
    batch_size = x_tensor.shape[0]
    class_detected_shap = target_s_[:batch_size, :]  # `target_s_`의 크기를 맞춤

    # 모델 실행
    selected_regions, _ = model(
        top_region_features=x_tensor, 
        class_detected=class_detected_shap,
        return_loss=False
    )
    return selected_regions.cpu().numpy()

# SHAP Explainer 생성
explainer = shap.DeepExplainer(model=shap_model_fn, data=X)

# SHAP 값 계산
shap_values = explainer.shap_values(X)  # [Batch, Feature_dim]

# 각 데이터 포인트의 가장 중요한 특성 출력
for i, sample_shap_values in enumerate(shap_values):
    important_feature_idx = np.argmax(np.abs(sample_shap_values))  # SHAP 값 절대값 기준
    print(f"Sample {i}: Most important feature index: {important_feature_idx}, SHAP value: {sample_shap_values[important_feature_idx]}")

# 시각화
shap.summary_plot(shap_values, X)


ModuleNotFoundError: No module named 'tensorflow'

In [88]:
import shap
import torch
import numpy as np

model.eval()

X = collected_features_train_.cpu().numpy()

def shap_model_fn(x):
    x_tensor = torch.tensor(x, dtype=torch.float32).to(next(model.parameters()).device)
    x_tensor = x_tensor.view(x_tensor.shape[0], 29, 1024)
    
    # `class_detected` 기본값을 사용하도록 호출
    _, selected_region_features = model(
        top_region_features=x_tensor,
        return_loss=False  # 필요 시만 설정
    )
    return selected_region_features.cpu().numpy()

explainer = shap.DeepExplainer(shap_model_fn, torch.tensor(X, dtype=torch.float32).to(next(model.parameters()).device))

shap_values = explainer.shap_values(torch.tensor(X, dtype=torch.float32).to(next(model.parameters()).device))

shap.summary_plot(shap_values, X)


ModuleNotFoundError: No module named 'tensorflow'

## nlp model

In [78]:
cp_nlp = []

for key in checkpoint['model'].keys():
    a = 'language_model'
    if a in key:
        cp_nlp.append(key)

In [80]:
cp_nlp

['language_model.gpt_with_lm_head.transformer.wte.weight',
 'language_model.gpt_with_lm_head.transformer.wpe.weight',
 'language_model.gpt_with_lm_head.transformer.h.0.ln_1.weight',
 'language_model.gpt_with_lm_head.transformer.h.0.ln_1.bias',
 'language_model.gpt_with_lm_head.transformer.h.0.attn.causal_mask',
 'language_model.gpt_with_lm_head.transformer.h.0.attn.mask_out_value',
 'language_model.gpt_with_lm_head.transformer.h.0.attn.c_attn.weight',
 'language_model.gpt_with_lm_head.transformer.h.0.attn.c_attn.bias',
 'language_model.gpt_with_lm_head.transformer.h.0.attn.c_proj.weight',
 'language_model.gpt_with_lm_head.transformer.h.0.attn.c_proj.bias',
 'language_model.gpt_with_lm_head.transformer.h.0.attn.uk.weight',
 'language_model.gpt_with_lm_head.transformer.h.0.attn.uk.bias',
 'language_model.gpt_with_lm_head.transformer.h.0.attn.uv.weight',
 'language_model.gpt_with_lm_head.transformer.h.0.attn.uv.bias',
 'language_model.gpt_with_lm_head.transformer.h.0.ln_2.weight',
 'langu

In [162]:
from typing import Optional, Tuple

import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss
from torchinfo import summary
from transformers import GPT2LMHeadModel
from transformers.generation_beam_search import BeamSearchScorer


class Conv1DWithTrainedWeights(nn.Module):
    """
    Same functionality as Conv1D class of transformers.pytorch_utils but allows initialization with trained weights.

    Conv1D has the same functionality as a linear layer.
    It transforms the inputted hidden_states from shape [batch x sequence_len x hidden_dim] to [batch x sequence_len x 3*hidden_dim],
    thus allowing the retrieval of the query, key and value matrices
    """

    def __init__(self, trained_weight, trained_bias):
        super(Conv1DWithTrainedWeights, self).__init__()
        self.weight = nn.Parameter(trained_weight, requires_grad=False)  # of shape [hidden_dim x 3*hidden_dim] for c_attn, of shape [hidden_dim x hidden_dim] for c_proj
        self.bias = nn.Parameter(trained_bias, requires_grad=False)  # of shape [3 * hidden_dim] for c_attn, of shape [hidden_dim] for c_proj

    def forward(self, x):  # x has shape [batch x sequence_len x hidden_dim]
        size_out = x.size()[:-1] + (self.weight.size(-1),)
        x = torch.addmm(self.bias, x.view(-1, x.size(-1)), self.weight)
        x = x.view(size_out)
        return x  # x has shape [batch x sequence_len x 3*hidden_dim] for c_attn, shape [batch x sequence_len x hidden_dim] for c_proj


class GPT2PseudoAttention(nn.Module):
    def __init__(
        self,
        c_attn_weights_and_bias: tuple[torch.FloatTensor],  # pre-trained weights and bias for retrieving query, key, value matrices
        c_proj_weights_and_bias: tuple[torch.FloatTensor],  # pre-trained weights and bias for projecting concatenated heads to original hidden dim
    ):

        super().__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.c_attn = Conv1DWithTrainedWeights(
            trained_weight=c_attn_weights_and_bias[0],
            trained_bias=c_attn_weights_and_bias[1],
        )
        self.c_proj = Conv1DWithTrainedWeights(
            trained_weight=c_proj_weights_and_bias[0],
            trained_bias=c_proj_weights_and_bias[1],
        )

        self.embed_dim = 1024
        self.num_heads = 16
        self.head_dim = self.embed_dim // self.num_heads
        self.split_size = self.embed_dim

        self.attn_dropout = nn.Dropout(p=0.1)
        self.resid_dropout = nn.Dropout(p=0.1)

        # seq_len can maximally be 1024 tokens
        max_positions = 1024

        # create a causal mask for masking out attention weights in the masked self-attention operator (masking out weights of tokens that lie ahead of the attended token)
        # first create a lower triangular matrix
        lower_triangular_matrix = torch.tril(torch.ones((max_positions, max_positions), dtype=torch.uint8))
        # then save lower_triangular_matrix (with additional dimensions for batch_size and num_heads) in a buffer
        # (to make sure the causal mask does not get updated during backprop)
        self.register_buffer("causal_mask", lower_triangular_matrix.view(1, 1, max_positions, max_positions))

        # value for masking out attention weights
        self.register_buffer("mask_out_value", torch.tensor(-1e4))

        # matrices for getting key and value matrices for image hidden states
        self.uk = nn.Linear(in_features=self.embed_dim, out_features=self.embed_dim)
        self.uv = nn.Linear(in_features=self.embed_dim, out_features=self.embed_dim)

    def _split_heads(self, tensor, num_heads, head_dim):
        """
        Splits hidden_dim (i.e. 1024) into num_heads (i.e. 16) and head_dim (i.e. 64)
        """
        new_shape = tensor.size()[:-1] + (num_heads, head_dim)
        tensor = tensor.view(new_shape)
        return tensor.permute(0, 2, 1, 3)  # (batch_size, num_heads, seq_len, head_dim)

    def _attn(self, query_word, key_image_word, value_image_word, attention_mask):
        attn_weights = torch.matmul(query_word, key_image_word.transpose(-1, -2))  # shape [batch_size x num_heads x seq_len x 1+seq_len]

        # scale attention weights
        attn_weights = attn_weights / (value_image_word.size(-1) ** 0.5)

        # create and apply the final causal mask to weights
        query_length, key_length = query_word.size(-2), key_image_word.size(-2)

        # note that this causal mask has a shape of seq_len x 1+seq_len (in the last 2 dims),
        # with the first column of the mask only consisting of True boolean values
        # meaning attention weights corresponding to images (which are stored in the first column) are not masked out!
        causal_mask = self.causal_mask[:, :, key_length - query_length: key_length, :key_length].to(torch.bool)

        # select the attention weights where the causal mask has True values, select -1e4 where the causal mask has False values
        attn_weights = torch.where(causal_mask, attn_weights, self.mask_out_value.to(attn_weights.dtype))

        # apply the attention mask of shape [batch_size, 1, 1, 1+seq_len] for masking out padding tokens
        # there is an additional column of zeros for the attention weights corresponding to the image,
        # such that these are not masked out
        attn_weights = attn_weights + attention_mask

        attn_weights = nn.functional.softmax(attn_weights, dim=-1)

        # downcast (if necessary) back to V's dtype (if in mixed-precision) -- no-op otherwise
        attn_weights = attn_weights.type(value_image_word.dtype)
        attn_weights = self.attn_dropout(attn_weights)

        attn_output = torch.matmul(attn_weights, value_image_word)  # shape [batch_size x num_heads x seq_len x head_dim]

        return attn_output,attn_weights

    def _merge_heads(self, tensor, num_heads, head_dim):
        """
        Merges num_heads (i.e. 16) and head_dim (i.e. 64) into hidden_dim (i.e. 1024)
        """
        tensor = tensor.permute(0, 2, 1, 3).contiguous()
        new_shape = tensor.size()[:-2] + (num_heads * head_dim,)
        return tensor.view(new_shape)

    def _merge_heads(self, tensor, num_heads, head_dim):
        """
        Merges num_heads and head_dim into hidden_dim
        """
        # Debug input tensor shape
        #print(f"[DEBUG] Input tensor shape before merge_heads: {tensor.shape}")
        #print(f"[DEBUG] num_heads: {num_heads}, head_dim: {head_dim}")
    
        if tensor.size(1) != num_heads:
            raise ValueError(f"[ERROR] Number of heads mismatch: expected {num_heads}, got {tensor.size(1)}")
    
        tensor = tensor.permute(0, 2, 1, 3).contiguous()  # (batch_size, seq_len, num_heads, head_dim)
        new_shape = tensor.size()[:-2] + (num_heads * head_dim,)  # (batch_size, seq_len, hidden_dim)
    
        #print(f"[DEBUG] Output tensor shape after merge_heads: {new_shape}")
        return tensor.view(new_shape)

    def forward(self,
                word_hidden_states,  # shape [batch_size x seq_len x hidden_dim]
                image_hidden_states,  # shape [batch_size x hidden_dim]
                attention_mask,  # shape [batch_size, 1, 1, 1+seq_len]
                layer_past,
                use_cache,output_attentions=True):

        # query, key, value matrices each have shape [batch_size x seq_len x hidden_dim]
        q_word, k_word, v_word = self.c_attn(word_hidden_states).split(self.split_size, dim=2)
        
        # if layer_past is None, we are either training the model or generating the first token in text generation mode
        if layer_past is None:
            # add an addition dimension to the image_hidden_states
            #print("1 image hidden state : ",image_hidden_states.shape)
            image_hidden_states = image_hidden_states[:, None, :]  # shape [batch_size x 1 x hidden_dim]

            # get the key and value matrices for the image hidden states
            k_image = self.uk(image_hidden_states)  # shape [batch_size x 1 x hidden_dim]
            #print('k_image',k_image.shape)
            #print("2 k image hidden state : ",k_image.shape)
            v_image = self.uv(image_hidden_states)  # shape [batch_size x 1 x hidden_dim]
            #print("3 v image hidden state : ",v_image.shape)
            # if the batch_size is different, then we are in beam search generation mode (adjust k and v image matrices accordingly)
            if k_image.size(0) != k_word.size(0):
                num_beams = k_word.size(0) // k_image.size(0)
                k_image = k_image.repeat_interleave(num_beams, dim=0)
                v_image = v_image.repeat_interleave(num_beams, dim=0)
            #print(f'image_hidden_states:{image_hidden_states}')
            #print(f'kword:{k_word.shape}')
            #print(f'k image:{k_image.shape}')
            #print("4 v image hidden state : ",v_image.shape)
            k_image_word = torch.cat((k_image, k_word), dim=1)  # shape [batch_size x 1+seq_len x hidden_dim]
           
            v_image_word = torch.cat((v_image, v_word), dim=1)  # shape [batch_size x 1+seq_len x hidden_dim]
            #print("5 v word hidden state : ",v_word.shape)
            #print("6 v image word hidden state : ",v_image_word.shape)
            q_word = self._split_heads(q_word, self.num_heads, self.head_dim)  # shape [batch_size x num_heads x seq_len x head_dim]
            k_image_word = self._split_heads(k_image_word, self.num_heads, self.head_dim)  # shape [batch_size x num_heads x 1+seq_len x head_dim]
            v_image_word = self._split_heads(v_image_word, self.num_heads, self.head_dim)  # shape [batch_size x num_heads x 1+seq_len x head_dim]
            #print(f"7 q word : {q_word.shape}, k img word : {k_image_word.shape}, v img word : {v_image_word.shape}")
            if use_cache is True:
                present = (k_image_word, v_image_word)
            else:
                present = None

            attn_output,attn_weights = self._attn(q_word, k_image_word, v_image_word, attention_mask)  # shape [batch_size x num_heads x seq_len x head_dim]
            #print(f"8 attn_output : {attn_output.shape}, attn_weights : {attn_weights.shape}")
        else:
            # if there is a layer_past (which stores key and value tensors of past tokens), then this means we are in text generation mode
            q_word = self._split_heads(q_word, self.num_heads, self.head_dim)  # shape [batch_size x num_heads x 1 x head_dim]
            k_word = self._split_heads(k_word, self.num_heads, self.head_dim)  # shape [batch_size x num_heads x 1 x head_dim]
            v_word = self._split_heads(v_word, self.num_heads, self.head_dim)  # shape [batch_size x num_heads x 1 x head_dim]

            past_key, past_value = layer_past
            k = torch.cat((past_key, k_word), dim=-2)
            v = torch.cat((past_value, v_word), dim=-2)

            present = (k, v)

            attn_output,attn_weights = self._attn(q_word, k, v, attention_mask)  # shape [batch_size x num_heads x seq_len x head_dim]

        attn_output = self._merge_heads(attn_output, self.num_heads, self.head_dim)  # shape [batch_size x seq_len x hidden_dim]
          # shape [batch_size x seq_len x hidden_dim]
        attn_output = self.c_proj(attn_output)
        attn_output = self.resid_dropout(attn_output)  # shape [batch_size x seq_len x hidden_dim]
        if output_attentions:
            #print("9 Attention shape: ",attn_weights.shape, attn_output.shape, self.num_heads, self.head_dim)
            #attn_weights = self._merge_heads(attn_weights, self.num_heads, self.head_dim)
            return attn_output,present,attn_weights
        return attn_output, present


class LanguageModel(nn.Module):
    """
    GPT2 model with a language modeling head and pseudo self-attention.

    Pseudo self-attention is based on the papar "Encoder-Agnostic Adaptation for Conditional Language Generation" (https://arxiv.org/abs/1908.06938).
    It is a technique to condition a pretrained language model to arbitrary conditional input (in my case features of chest x-ray images).

    The code is largely the same as the GPT2 implementation by Huggingface (https://github.com/huggingface/transformers/blob/d0acc9537829e7d067edbb791473bbceb2ecf056/src/transformers/models/gpt2/modeling_gpt2.py),
    except for the custom GPT2PseudoAttention class replacing the GPT2Attention class.

    Recommended reading to understand the GPT2 source code: https://amaarora.github.io/2020/02/18/annotatedGPT2.html
    """

    def __init__(self):
        super().__init__()
        self.checkpoint = "healx/gpt-2-pubmed-medium"
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.bos_token_id = 50256
        self.eos_token_id = 50256
        self.pad_token_id = 50256

        # use GPT2 model with language modeling head, since we want to generate phrases
        self.gpt_with_lm_head = GPT2LMHeadModel.from_pretrained(self.checkpoint)

        # freeze all parameters of the model
        for param in self.gpt_with_lm_head.parameters():
            param.requires_grad = False

        # replace normal attention layers by pseudo attention layers
        self._replace_attention_by_pseudo_attention()

        # divide model into GPT part and language modeling head part
        self.gpt = self.gpt_with_lm_head.transformer
        self.lm_head = self.gpt_with_lm_head.lm_head

        # divide GPT part into word embedding layer, positional embedding layer, dropout layer, gpt2 blocks and final layernorm
        gpt_children = list(self.gpt.children())
        self.wte = gpt_children[0]  # word embedding layer
        self.wpe = gpt_children[1]  # positional embedding layer
        self.drop = gpt_children[2]  # dropout layer
        self.gpt2_blocks = gpt_children[3]  # type: nn.ModuleList
        self.final_layernorm = gpt_children[4]

        # convert each individual gpt2_block into a nn.ModuleList
        self.gpt2_blocks = nn.ModuleList(nn.ModuleList(gpt2_block.children()) for gpt2_block in self.gpt2_blocks)

        # small neural network to transform embeddings coming from the image feature space into embeddings in the text feature space
        self.feature_space_transformation_nn = nn.Sequential(
            nn.Linear(in_features=1024, out_features=1024),
            nn.ReLU(),
            nn.Linear(in_features=1024, out_features=1024)
        )

    def _replace_attention_by_pseudo_attention(self):
        GPT2PSA_list = []

        for gpt2_block in self.gpt_with_lm_head.transformer.h:
            # extract trained weights and biases
            attn = gpt2_block.attn
            c_attn_weights = attn.c_attn.weight.detach()
            c_attn_bias = attn.c_attn.bias.detach()
            c_proj_weights = attn.c_proj.weight.detach()
            c_proj_bias = attn.c_proj.bias.detach()

            # initialize GPT2PseudoAttention module
            GPT2PSA = GPT2PseudoAttention(
                c_attn_weights_and_bias=(c_attn_weights, c_attn_bias),
                c_proj_weights_and_bias=(c_proj_weights, c_proj_bias),
            )

            GPT2PSA_list.append(GPT2PSA)

        for i, GPT2PSA in enumerate(GPT2PSA_list):
            self.gpt_with_lm_head.transformer.h[i].attn = GPT2PSA

    def forward(self,
                input_ids: torch.LongTensor,  # shape [batch_size x seq_len]
                attention_mask: torch.FloatTensor,  # shape [batch_size x seq_len]
                image_hidden_states: torch.FloatTensor,  # shape [batch_size x image_hidden_dim] (with image_hidden_dim = 1024)
                return_loss: bool = False,
                past_key_values: Optional[Tuple[Tuple[torch.Tensor]]] = None,
                position_ids: Optional[torch.LongTensor] = None,
                use_cache: Optional[bool] = False,
                output_attentions: bool = True,
                ):
        """
        If return_loss is True, returns the language modeling loss.
        If return_loss is False (in which we are in text generation mode and use_cache will be True), returns the language modeling logits (of shape batch_size x seq_len x vocab_size)
        as well as the so-called presents (which store the key and value tensors of the previous tokens, such that they don't have to be recomputed every time during text generation).

        To compute the loss, the input_ids are used as labels.
        To prevent padding tokens from counting towards the loss, the attention_mask is transformed to a boolean mask and inverted.
        Then this inverted boolean mask is used to set all padding token ids to -100.
        In the cross entropy loss, the ignore_index is set to -100, such that padding token ids are ignored as targets.

        Furthermore, the label at the first position of the sequence is discarded and the labels are shifted accordingly (i.e. one to the left),
        such that the language modeling logits align with the labels that they are trying to predict.
        """
        # get a boolean copy of the attention_mask and invert it
        mask_to_ignore_padding_tokens_for_loss_computation = ~(attention_mask.to(torch.bool))

        # transform image_hidden_states from image feature space to text feature space
        image_hidden_states = self.feature_space_transformation_nn(image_hidden_states)  # shape [batch_size x word_hidden_dim], with word_hidden_dim = 1024

        input_shape = input_ids.size()
        input_ids = input_ids.view(-1, input_shape[-1])
        batch_size = input_ids.shape[0]

        # pass the token ids through the word embedding layer to get the word embeddings
        inputs_embeds = self.wte(input_ids)  # shape [batch_size x seq_len x hidden_dim]

        # position_ids is a tensor that specifies the position of each token in the input (necessary to create positional embeddings)
        if position_ids is not None:
            position_ids = position_ids.view(-1, input_shape[-1])

        if past_key_values is None:
            past_length = 0
            past_key_values = tuple([None] * len(self.gpt2_blocks))
        else:
            past_length = past_key_values[0][0].size(-2)
        if position_ids is None:
            position_ids = torch.arange(past_length, input_shape[-1] + past_length, dtype=torch.long, device=self.device)
            position_ids = position_ids.unsqueeze(0).view(-1, input_shape[-1])  # shape [1 x seq_len]

        # pass the position ids through the positional embedding layer to get the positional embeddings
        position_embeds = self.wte(position_ids)  # shape [1 x seq_len x hidden_dim]

        # addition is broadcasted around batch_size dimension
        word_hidden_states = inputs_embeds + position_embeds  # shape [batch_size x seq_len x hidden_dim]

        word_hidden_states = self.drop(word_hidden_states)

        output_shape = input_shape + (word_hidden_states.size(-1),)

        # we change the attention_mask shape to [batch_size, 1, 1, seq_len], since the attention_mask is later applied to the last dimension of
        # the attention weights that are of shape [batch_size x num_heads x seq_len x 1+seq_len]
        attention_mask = attention_mask.view(batch_size, -1)
        attention_mask = attention_mask[:, None, None, :]

        # since we have 1 additional column in the attention weights (i.e. 1+seq_len in the last dimension) due to the additional concatenated key matrix
        # of the image hidden states (see forward method of GPT2PseudoAttention), we have to shift the attention mask "one to the right" and add a column of ones
        # to the left such that the attention weights corresponding to the image are not masked out
        attention_mask_size = attention_mask.size()
        ones_column = torch.ones(attention_mask_size[:-1] + (1,), dtype=torch.int64, device=self.device)  # shape [batch_size, 1, 1, 1]
        attention_mask = torch.cat((ones_column, attention_mask), dim=-1)  # shape [batch_size, 1, 1, 1+seq_len]

        # Since attention_mask is 1.0 for positions we want to attend and 0.0 for
        # masked positions, this operation will create a tensor which is 0.0 for
        # positions we want to attend and -10000.0 for masked positions.
        # Since we are adding it to the raw scores before the softmax, this is
        # effectively the same as removing these entirely
        attention_mask = attention_mask.to(dtype=next(self.parameters()).dtype)  # dtype should be either torch.float32 or torch.float16
        attention_mask = (1.0 - attention_mask) * -10000.0

        presents = () if use_cache else None
        attentions = []
        for gpt2_block, layer_past in zip(self.gpt2_blocks, past_key_values):
            layer_norm_1 = gpt2_block[0]
            pseudo_self_attention = gpt2_block[1]
            layer_norm_2 = gpt2_block[2]
            mlp = gpt2_block[3]

            residual = word_hidden_states
            word_hidden_states = layer_norm_1(word_hidden_states)

            word_hidden_states, present, attention_weights = pseudo_self_attention(word_hidden_states, image_hidden_states, attention_mask, layer_past, use_cache)
            attentions.append(attention_weights)#youna add
            # residual connection
            word_hidden_states = word_hidden_states + residual

            residual = word_hidden_states
            word_hidden_states = layer_norm_2(word_hidden_states)
            word_hidden_states = mlp(word_hidden_states)

            # residual connection
            word_hidden_states = word_hidden_states + residual

            if use_cache:
                presents += (present,)

        word_hidden_states = self.final_layernorm(word_hidden_states)

        word_hidden_states = word_hidden_states.view(output_shape)

        lm_logits = self.lm_head(word_hidden_states)  # shape [batch_size x seq_len x vocab_size], with vocab_size = 50257

        if return_loss:
            # use input_ids as ground_truth labels
            labels = input_ids

            # set padding tokens to -100, such that they are ignored and don't count towards the loss
            labels[mask_to_ignore_padding_tokens_for_loss_computation] = -100

            # shift the tokens, i.e. discard the last token in the sequence for the logits,
            # and discard the first token in the sequence for the labels

            # this way, the logits of the first token are "aligned" with the second token label,
            # the logits of the second token are "aligned" with the third token label, and so on...
            # since the previous token should predict the next token

            # discard the last lm_logit corresponding to the last token
            shift_logits = lm_logits[:, :-1, :].contiguous()  # shape [batch_size x seq_len-1 x vocab_size]

            # discard the first token in the sequence
            shift_labels = labels[:, 1:].contiguous()  # shape [batch_size x seq_len-1]

            # flatten the tokens
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))  # shape [batch_size*seq_len-1 x vocab_size]
            shift_labels = shift_labels.view(-1)  # shape [batch_size * seq_len-1]

            # padding tokens are ignored for loss computation, and loss is averaged over non-ignored targets
            loss_fct = CrossEntropyLoss(ignore_index=-100)
            loss = loss_fct(shift_logits, shift_labels)

            return loss
        if output_attentions:
            return lm_logits, presents,attentions
        if use_cache:
            return lm_logits, presents

    @torch.no_grad()
    def generate(self,
                 image_hidden_states: torch.FloatTensor,  # shape [batch_size x image_hidden_dim]
                 max_length: int = None,
                 num_beams: int = 1,
                 num_beam_groups: int = 1,
                 do_sample: bool = False,
                 num_return_sequences: int = 1,
                 early_stopping: bool = False
                 ) -> torch.LongTensor:  # shape [batch_size x longest_generated_sequence_length]
        """
        Generates output ids for a batch of image features.
        These output ids can then be decoded by the tokenizer to get the generated sentences.
        """
        batch_size = image_hidden_states.size(0)

        # start with the bos_token_id for all image features in the batch.
        input_ids = torch.full(size=(batch_size, 1), fill_value=self.bos_token_id, dtype=torch.int64, device=self.device)
        model_kwargs = {"attention_mask": torch.ones(size=(batch_size, 1), dtype=torch.int64, device=self.device),
                        "use_cache": True}

        is_greedy_gen_mode = (num_beams == 1) and (num_beam_groups == 1) and do_sample is False
        is_sample_gen_mode = (num_beams == 1) and (num_beam_groups == 1) and do_sample is True
        is_beam_gen_mode = (num_beams > 1) and (num_beam_groups == 1) and do_sample is False
        is_beam_sample_gen_mode = (num_beams > 1) and (num_beam_groups == 1) and do_sample is True
        is_group_beam_gen_mode = (num_beams > 1) and (num_beam_groups > 1)

        if num_beam_groups > num_beams:
            raise ValueError("'num_beam_groups' has to be smaller or equal to 'num_beams'")
        if is_group_beam_gen_mode and do_sample is True:
            raise ValueError(
                "Diverse beam search cannot be used in sampling mode. Make sure that 'do_sample' is set to 'False'."
            )

        # go into different generation modes
        if is_greedy_gen_mode:
            if num_return_sequences > 1:
                raise ValueError(
                    f"num_return_sequences has to be 1, but is {num_return_sequences} when doing greedy search."
                )

            return self.greedy_search(
                input_ids,
                image_hidden_states,
                max_length,
                **model_kwargs
            )
        elif is_sample_gen_mode:
            raise NotImplementedError("Multinomial sampling is not implemented.")
        elif is_beam_gen_mode:
            if num_return_sequences > num_beams:
                raise ValueError("'num_return_sequences' has to be smaller or equal to 'num_beams'.")

            if max_length is None:
                raise ValueError("max_length has to be set for beam generation.")

            beam_scorer = BeamSearchScorer(
                batch_size=batch_size,
                num_beams=num_beams,
                device=self.device,
                length_penalty=1.0,  # length_penalty > 0.0 encourages the model to generate shorter sequences
                do_early_stopping=early_stopping,
                num_beam_hyps_to_keep=num_return_sequences,
            )

            # interleave input_ids with 'num_beams' additional sequences per batch
            input_ids, model_kwargs = self._expand_inputs_for_generation(input_ids, expand_size=num_beams, **model_kwargs)

            return self.beam_search(
                input_ids,
                image_hidden_states,
                max_length,
                beam_scorer,
                **model_kwargs,
            )
        elif is_beam_sample_gen_mode:
            raise NotImplementedError("Beam-search multinomial sampling is not implemented.")
        elif is_group_beam_gen_mode:
            raise NotImplementedError("Diverse beam-search decoding is not implemented.")

    def _expand_inputs_for_generation(self, input_ids, expand_size, attention_mask, **model_kwargs):
        expanded_return_idx = (
            torch.arange(input_ids.shape[0]).view(-1, 1).repeat(1, expand_size).view(-1).to(input_ids.device)
        )
        input_ids = input_ids.index_select(0, expanded_return_idx)

        if attention_mask is not None:
            model_kwargs["attention_mask"] = attention_mask.index_select(0, expanded_return_idx)

        return input_ids, model_kwargs

    def _reorder_cache(self, past, beam_idx):
        return tuple(
            tuple(past_state.index_select(0, beam_idx.to(past_state.device)) for past_state in layer_past)
            for layer_past in past
        )

    def prepare_inputs_for_generation(self, input_ids, past=None, **kwargs):
        # only use last token for inputs_ids if past is defined in kwargs
        if past:
            input_ids = input_ids[:, -1].unsqueeze(-1)

        attention_mask = kwargs.get("attention_mask", None)
        position_ids = kwargs.get("position_ids", None)

        if attention_mask is not None and position_ids is None:
            # create position_ids on the fly for batch generation
            position_ids = attention_mask.long().cumsum(-1) - 1
            position_ids.masked_fill_(attention_mask == 0, 1)
            if past:
                position_ids = position_ids[:, -1].unsqueeze(-1)
        else:
            position_ids = None
        return {
            "input_ids": input_ids,
            "past_key_values": past,
            "use_cache": kwargs.get("use_cache"),
            "position_ids": position_ids,
            "attention_mask": attention_mask,
        }

    def _update_model_kwargs_for_generation(self, presents, model_kwargs):
        model_kwargs["past"] = presents
        attention_mask = model_kwargs["attention_mask"]
        model_kwargs["attention_mask"] = torch.cat([attention_mask, attention_mask.new_ones((attention_mask.shape[0], 1))], dim=-1)

        return model_kwargs

    def beam_search(self,
                    input_ids,
                    image_hidden_states,
                    max_length,
                    beam_scorer,
                    **model_kwargs):
        batch_size = len(beam_scorer._beam_hyps)
        num_beams = beam_scorer.num_beams

        batch_beam_size, cur_len = input_ids.shape

        if num_beams * batch_size != batch_beam_size:
            raise ValueError(
                f"Batch dimension of 'input_ids' should be {num_beams * batch_size}, but is {batch_beam_size}."
            )

        beam_scores = torch.zeros((batch_size, num_beams), dtype=torch.float, device=input_ids.device)
        beam_scores[:, 1:] = -1e9
        beam_scores = beam_scores.view((batch_size * num_beams,))

        while True:
            model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)

            lm_logits, presents = self.forward(**model_inputs, image_hidden_states=image_hidden_states, return_loss=False)

            next_token_logits = lm_logits[:, -1, :]

            next_token_scores = nn.functional.log_softmax(next_token_logits, dim=-1)  # (batch_size * num_beams, vocab_size)

            next_token_scores = next_token_scores + beam_scores[:, None].expand_as(next_token_scores)

            vocab_size = next_token_scores.shape[-1]
            next_token_scores = next_token_scores.view(batch_size, num_beams * vocab_size)

            next_token_scores, next_tokens = torch.topk(
                next_token_scores, 2 * num_beams, dim=1, largest=True, sorted=True
            )

            next_indices = torch.div(next_tokens, vocab_size, rounding_mode="floor")
            next_tokens = next_tokens % vocab_size

            beam_outputs = beam_scorer.process(
                input_ids,
                next_token_scores,
                next_tokens,
                next_indices,
                pad_token_id=self.pad_token_id,
                eos_token_id=self.eos_token_id,
                # beam_indices=None,
            )

            beam_scores = beam_outputs["next_beam_scores"]
            beam_next_tokens = beam_outputs["next_beam_tokens"]
            beam_idx = beam_outputs["next_beam_indices"]

            input_ids = torch.cat([input_ids[beam_idx, :], beam_next_tokens.unsqueeze(-1)], dim=-1)

            model_kwargs = self._update_model_kwargs_for_generation(presents, model_kwargs)

            if model_kwargs["past"] is not None:
                model_kwargs["past"] = self._reorder_cache(model_kwargs["past"], beam_idx)

            # increase cur_len
            cur_len += 1

            if beam_scorer.is_done or (max_length and cur_len >= max_length):
                break

        sequence_outputs = beam_scorer.finalize(
            input_ids,
            beam_scores,
            next_tokens,
            next_indices,
            pad_token_id=self.pad_token_id,
            eos_token_id=self.eos_token_id,
            max_length=max_length,
        )

        return sequence_outputs["sequences"]

    def greedy_search(self,
                      input_ids,  # shape [batch_size x seq_len]
                      image_hidden_states,  # shape [batch_size x image_hidden_dim]
                      max_length,
                      **model_kwargs
                      ) -> torch.LongTensor:  # shape [batch_size x longest_generated_sequence_length]
        batch_size = input_ids.size(0)
        seq_len = input_ids.size(1)

        # keep track of which sequences are already finished
        # a 1 denotes that a sentence in a batch is unfinished, a 0 denotes that a sentence has finished
        # finished sentences are padded until all sentences in the batch are finished
        unfinished_sequences = torch.ones(size=(batch_size,), dtype=torch.int64, device=self.device)
        cur_len = seq_len

        while True:
            model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)

            lm_logits, presents = self.forward(**model_inputs, image_hidden_states=image_hidden_states, return_loss=False)

            next_token_logits = lm_logits[:, -1, :]  # of shape [batch_size x vocab_size]

            # no need to convert logits into probabilities first (via softmax), argmax can be directly applied to logits
            next_tokens = torch.argmax(next_token_logits, dim=-1)  # of shape [batch_size]

            # convert next token to padding token if given sentence has already finished (denoted by a 0 in unfinished_sequences)
            # padding tokens are ignored when decoding, if skip_special_tokens=True is set
            next_tokens = next_tokens * unfinished_sequences + self.pad_token_id * (1 - unfinished_sequences)

            # update variables for next step
            input_ids = torch.cat([input_ids, next_tokens[:, None]], dim=-1)
            model_kwargs = self._update_model_kwargs_for_generation(presents, model_kwargs)
            cur_len += 1

            # if eos_token was found in one sentence, set sentence to finished (by converting 1 to 0 for that sentence)
            binary_mask = (next_tokens != self.eos_token_id).long()
            unfinished_sequences = unfinished_sequences.mul(binary_mask)

            # stop when all sentences are finished (i.e. all sentences have value 0 in unfinished_sequences),
            # or if we exceed the maximum length
            if unfinished_sequences.max() == 0 or (max_length and cur_len >= max_length):
                break

        return input_ids


def print_model_summary(batch_size, seq_len, verbose):
    """
    Choose between:
        verbose = 0 (only model params)
        verbose = 1 (model params and output shape of batch)
        verbose = 2 (model params and output shape of batch, more detailed)
    """
    inputs = {}
    inputs["input_ids"] = torch.randint(low=0, high=50257, size=(batch_size, seq_len))
    inputs["attention_mask"] = torch.randint(low=0, high=2, size=(batch_size, seq_len))
    inputs["image_hidden_states"] = torch.rand(batch_size, (2048 * 8 * 8))

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = LanguageModel()
    model.to(device, non_blocking=True)

    inputs = {k: v.to(device, non_blocking=True) for k, v in inputs.items()}

    if verbose == 0:
        summary(model)
    else:
        summary(model, input_data=dict(inputs), verbose=verbose)


In [177]:
#torch.cuda.empty_cache()

In [163]:

model_nlp = LanguageModel().to(device)

# Filter and load model weights
nlp_state_dict = {
    key.replace("language_model.gpt_with_lm_head.", ""): checkpoint["model"][key]
    for key in cp_nlp  # Only keys in cp_nlp
}

# Load the filtered state_dict into the model
model_nlp.gpt_with_lm_head.load_state_dict(nlp_state_dict, strict=False)

C:\Users\seclab\anaconda3\envs\rgrg_env\lib\site-packages\transformers\modeling_utils.py:392: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file

_IncompatibleKeys(missing_keys=[], unexpected_keys=['language_model.gpt.wte.weight', 'language_model.gpt.wpe.weight', 'language_model.gpt.h.0.ln_1.weight', 'language_model.gpt.h.0.ln_1.bias', 'language_model.gpt.h.0.attn.causal_mask', 'language_model.gpt.h.0.attn.mask_out_value', 'language_model.gpt.h.0.attn.c_attn.weight', 'language_model.gpt.h.0.attn.c_attn.bias', 'language_model.gpt.h.0.attn.c_proj.weight', 'language_model.gpt.h.0.attn.c_proj.bias', 'language_model.gpt.h.0.attn.uk.weight', 'language_model.gpt.h.0.attn.uk.bias', 'language_model.gpt.h.0.attn.uv.weight', 'language_model.gpt.h.0.attn.uv.bias', 'language_model.gpt.h.0.ln_2.weight', 'language_model.gpt.h.0.ln_2.bias', 'language_model.gpt.h.0.mlp.c_fc.weight', 'language_model.gpt.h.0.mlp.c_fc.bias', 'language_model.gpt.h.0.mlp.c_proj.weight', 'language_model.gpt.h.0.mlp.c_proj.bias', 'language_model.gpt.h.1.ln_1.weight', 'language_model.gpt.h.1.ln_1.bias', 'language_model.gpt.h.1.attn.causal_mask', 'language_model.gpt.h.1.

In [164]:
# checkpoint에서 필요한 키만 필터링 및 이름 매핑
nlp_state_dict = {}
for key, value in checkpoint["model"].items():
    if key.startswith("language_model.gpt_with_lm_head."):
        new_key = key.replace("language_model.gpt_with_lm_head.", "")  # 키 이름 매핑
        nlp_state_dict[new_key] = value

# 필터링된 state_dict를 로드
missing_keys, unexpected_keys = model_nlp.gpt_with_lm_head.load_state_dict(nlp_state_dict, strict=False)

# 로드 결과 확인
print(f"Missing keys: {missing_keys}")
print(f"Unexpected keys: {unexpected_keys}")


Missing keys: []
Unexpected keys: []


In [165]:
model_nlp.eval()
# Tokenize the input

LanguageModel(
  (gpt_with_lm_head): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50257, 1024)
      (wpe): Embedding(1024, 1024)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-23): 24 x GPT2Block(
          (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2PseudoAttention(
            (c_attn): Conv1DWithTrainedWeights()
            (c_proj): Conv1DWithTrainedWeights()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
            (uk): Linear(in_features=1024, out_features=1024, bias=True)
            (uv): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          

In [222]:
# selected_indices = torch.randint(0, 29, (region_features.size(0),))  # [batch_size]
# region_features_reduced = region_features[torch.arange(region_features.size(0)), selected_indices]

In [87]:
from transformers import GPT2Tokenizer

In [88]:
def get_tokenizer():
    checkpoint = "healx/gpt-2-pubmed-medium"
    tokenizer = GPT2Tokenizer.from_pretrained(checkpoint)
    tokenizer.pad_token = tokenizer.eos_token

    return tokenizer
tokenizer = get_tokenizer()

In [109]:
from full_model.train_full_model import get_tokenized_datasets
tokenized_train_dataset, tokenized_val_dataset = get_tokenized_datasets(tokenizer, raw_train_dataset, raw_val_dataset)

train_transforms = get_transforms("train")
val_transforms = get_transforms("val")

NameError: name 'raw_train_dataset' is not defined

In [151]:
from full_model.train_full_model import get_tokenized_datasets
tokenized_test_dataset, tokenized_test_dataset = get_tokenized_datasets(tokenizer, raw_test_dataset, raw_test_dataset)

train_transforms = get_transforms("train")
val_transforms = get_transforms("val")

100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 64.97ex/s]


In [ ]:
from src.full_model.custom_dataset import CustomDataset

train_dataset_complete = CustomDataset("train", tokenized_train_dataset, train_transforms, log)
val_dataset_complete = CustomDataset("val", tokenized_val_dataset, val_transforms, log)


In [152]:
from src.full_model.custom_dataset import CustomDataset

test_dataset_complete = CustomDataset("train", tokenized_test_dataset, train_transforms, log)
#val_dataset_complete = CustomDataset("val", tokenized_test_dataset, val_transforms, log)


In [144]:
tokenized_test_dataset[0]["bbox_coordinates"]

'[[144, 315, 512, 512], [315, 360, 512, 512], [433, 315, 512, 512], [180, 144, 512, 512]]'

In [153]:
from src.full_model.train_full_model import get_data_loaders

train_loader, val_loader = get_data_loaders(tokenizer, test_dataset_complete, test_dataset_complete)


In [155]:
i = 0 
for num_batch, batch in tqdm(enumerate(train_loader)):
    print(batch)
    images = batch["images"]
    
    image_targets = batch["image_targets"]

    region_has_sentence = batch["region_has_sentence"]
    region_is_abnormal = batch["region_is_abnormal"]

    batch_size = images.size(0)

    images = images.to(device, non_blocking=True)
    image_targets = [{k: v.to(device, non_blocking=True) for k, v in t.items()} for t in image_targets]
    region_has_sentence = region_has_sentence.to(device, non_blocking=True)
    region_is_abnormal = region_is_abnormal.to(device, non_blocking=True)
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]

    input_ids = input_ids.to(device, non_blocking=True)
    attention_mask = attention_mask.to(device, non_blocking=True)
    #obj_detector_loss_dict, top_region_features, class_detected = self.object_detector(images, image_targets)
    class_detected = []
   
    for target in batch["image_targets"]:
        indices = target["labels"]  # Get indices
        # Create a one-hot vector for the current indices
        one_hot = torch.zeros((len(indices),29), dtype=torch.bool)
        one_hot.scatter_(1, indices.unsqueeze(1), True)
        class_detected.append(one_hot)
    
    # Combine all one-hot vectors for the batch
    class_detected = torch.cat(class_detected, dim=0)
    top_region_features = object_test_f
    break





0it [00:00, ?it/s]

{'input_ids': tensor([[50256,    43,  2150,  ...,  1775,    13, 50256],
        [50256, 50256, 50256,  ..., 50256, 50256, 50256],
        [50256,  1858,   318,  ..., 50256, 50256, 50256],
        ...,
        [50256, 50256, 50256,  ..., 50256, 50256, 50256],
        [50256, 50256, 50256,  ..., 50256, 50256, 50256],
        [50256, 50256, 50256,  ..., 50256, 50256, 50256]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 0,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 0,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]]), 'images': tensor([[[[0.2064, 0.2194, 0.2324,  ..., 0.6869, 0.6219, 0.6998],
          [0.2713, 0.2583, 0.2454,  ..., 0.6609, 0.6609, 0.8167],
          [0.3492, 0.2843, 0.2194,  ..., 0.2973, 0.4272, 0.6869],
          ...,
          [0.2713, 0.2973, 0.3363,  ..., 0.7258, 0.7648, 0.7907],
          [0.3103, 0.3363, 0.3752,  ..., 0.7518, 0.7907, 0.8167],
          [0.3363, 0.3622, 0.40

In [156]:
def get_valid_decoder_input_for_training(
        class_detected,  # shape [batch_size x 29]
        region_has_sentence,  # shape [batch_size x 29]
        input_ids,  # shape [(batch_size * 29) x seq_len]
        attention_mask,  # shape [(batch_size * 29) x seq_len]
        region_features,  # shape [batch_size x 29 x 1024]
    ):
        """
        We want to train the decoder only on region features (and corresponding input_ids/attention_mask) whose corresponding sentences are non-empty and
        that were detected by the object detector.
        """
        # valid is of shape [batch_size x 29]
        device = class_detected.device#iadd
        region_has_sentence = region_has_sentence.to(device)#i add
    
        #valid = torch.logical_and(class_detected, region_has_sentence) # youna have to change
        valid =  region_has_sentence
        # reshape to [(batch_size * 29)], such that we can apply the mask to input_ids and attention_mask
        valid_reshaped = valid.reshape(-1)
        print(valid.shape)
        print(valid_reshaped.shape)
        valid_input_ids = input_ids[valid_reshaped]  # of shape [num_detected_regions_with_non_empty_gt_phrase_in_batch x seq_len]
        valid_attention_mask = attention_mask[valid_reshaped]  # of shape [num_detected_regions_with_non_empty_gt_phrase_in_batch x seq_len]
        valid_region_features = region_features[valid]  # of shape [num_detected_regions_with_non_empty_gt_phrase_in_batch x 1024]

        return valid_input_ids, valid_attention_mask, valid_region_features

def get_valid_decoder_input_for_evaluation(
    selected_regions,  # shape [batch_size x 29]
    input_ids,  # shape [(batch_size * 29) x seq_len]
    attention_mask  # shape [(batch_size * 29) x seq_len]
):
    """
    For evaluation, we want to evaluate the decoder on the top_region_features selected by the classifier to get a sentence generated.
    We also have to get the corresponding input_ids and attention_mask accordingly.
    """
    # reshape to [(batch_size * 29)]
    selected_regions = selected_regions.reshape(-1)

    valid_input_ids = input_ids[selected_regions]  # of shape [num_regions_selected_in_batch x seq_len]
    valid_attention_mask = attention_mask[selected_regions]  # of shape [num_regions_selected_in_batch x seq_len]

    return valid_input_ids, valid_attention_mask


In [82]:
valid_region_features.shape

torch.Size([74, 1024])

In [78]:
top_region_features[:8].shape

(8, 29, 1024)

In [48]:
top_region_features[0].shape

(8, 29, 1024)

In [158]:
print(len(region_has_sentence))

1


In [159]:
region_has_sentence

tensor([[ True, False,  True,  True,  True, False,  True, False,  True, False,
         False, False,  True, False,  True, False, False, False, False, False,
         False, False, False, False,  True, False, False, False, False]],
       device='cuda:0')

In [160]:
valid_input_ids, valid_attention_mask, valid_region_features = get_valid_decoder_input_for_training(
                 class_detected, region_has_sentence, input_ids, attention_mask, top_region_features[0]#top_region_features[:8]
             )

#valid_input_ids, valid_attention_mask = get_valid_decoder_input_for_evaluation(selected_regions, input_ids, attention_mask)


torch.Size([1, 29])
torch.Size([29])


In [161]:
# input_ids = torch.zeros((batch_size, 1024), dtype=torch.int64).to(device)

# # 모든 토큰에 대해 attention 활성화
# attention_mask = torch.ones((batch_size, 1024), dtype=torch.float32).to(device)


# with torch.no_grad():
#     output, attentions = model_nlp(
#         input_ids=input_ids,#input_ids
#         attention_mask=attention_mask,#attention_mask
#         image_hidden_states=region_features[0].unsqueeze(0), # regions_feature
#         output_attentions=True, # i add # return_loss , pask_key_values, position_ids, use_cache
#     )

model_dtype = next(model_nlp.parameters()).dtype  # Get model's parameter dtype
valid_region_features = torch.from_numpy(valid_region_features).to(dtype=model_dtype, device=device)

language_model_loss, pre,attn_output = model_nlp(
            valid_input_ids,
            valid_attention_mask,
            valid_region_features,
            False,#return_loss,
            None,#past_key_values=
            None,#position_ids
            True)##use_cache)
# 중요도 계산
#aggregated_attention = torch.stack(attentions).mean(dim=0)  # 평균화
#print("Aggregated Attention:", aggregated_attention)


In [166]:
# input_ids = torch.zeros((batch_size, 1024), dtype=torch.int64).to(device)

# # 모든 토큰에 대해 attention 활성화
# attention_mask = torch.ones((batch_size, 1024), dtype=torch.float32).to(device)


# with torch.no_grad():
#     output, attentions = model_nlp(
#         input_ids=input_ids,#input_ids
#         attention_mask=attention_mask,#attention_mask
#         image_hidden_states=region_features[0].unsqueeze(0), # regions_feature
#         output_attentions=True, # i add # return_loss , pask_key_values, position_ids, use_cache
#     )

#model_dtype = next(model_nlp.parameters()).dtype  # Get model's parameter dtype
#valid_region_features = torch.from_numpy(valid_region_features).to(dtype=model_dtype, device=device)

language_model_loss, pre,attentions = model_nlp(
            valid_input_ids,
            valid_attention_mask,
            valid_region_features,
            False,#return_loss,
            None,#past_key_values=
            None,#position_ids
            True)##use_cache)
# 중요도 계산
#aggregated_attention = torch.stack(attentions).mean(dim=0)  # 평균화
#print("Aggregated Attention:", aggregated_attention)


In [167]:
# 1. 이미지 Attention Weight 추출
image_attention_weights = attentions[-1][:, :, :, 0]  # [Batch, Heads, Query=67]

# 2. Heads 방향으로 평균화
avg_image_attention_weights = image_attention_weights.mean(dim=1)  # [Batch=74, Query=67]

# 3. 이미지 Region Feature 추출
image_features = attn_output[-1][:, -1, :]  # [Batch=74, Hidden_Dim=1024]

# 4. Query별 중요도를 Hidden_Dim에 적용
# 각 Query(67)의 중요도를 Hidden_Dim(1024)에 곱하도록 차원 변경
query_importance = (avg_image_attention_weights.unsqueeze(-1) * image_features.unsqueeze(1)).sum(dim=1)  # [Batch=74, Hidden_Dim=1024]


In [57]:
print(image_attention_weights.shape)
print(avg_image_attention_weights.shape)
print(image_features.shape)

torch.Size([79, 16, 80])
torch.Size([79, 80])
torch.Size([79, 1024])


In [168]:
query_importance.shape

torch.Size([9, 1024])

In [58]:
query_importance

tensor([[ 0.0560,  0.0327,  0.0052,  ...,  0.0751, -0.0744,  0.0991],
        [ 0.0210,  0.0150,  0.0003,  ...,  0.0009, -0.0168,  0.0035],
        [ 0.0071, -0.0010,  0.0064,  ...,  0.0022, -0.0070,  0.0050],
        ...,
        [ 0.0010, -0.0033,  0.0254,  ...,  0.0216, -0.0167,  0.0082],
        [-0.0037, -0.0028,  0.0280,  ...,  0.0236, -0.0186,  0.0078],
        [ 0.0171, -0.0184,  0.0216,  ...,  0.0117, -0.0287,  0.0254]],
       device='cuda:0', grad_fn=<SumBackward1>)

In [169]:
# Tensor를 NumPy 배열로 변환
query_importance_numpy = query_importance.detach().cpu().numpy()


# NumPy 배열을 저장
np.save("query_importance_test.npy", query_importance_numpy)



In [ ]:
torch.cuda.empty_cache()

In [ ]:
with torch.no_grad():
    output,attentnions = model_nlp.generate(
        image_hidden_states=region_features_reduced,
        max_length=300,  # Maximum length of the generated sequence
        num_beams=4,
        num_beam_groups=1,# Beam search for better results
        do_sample=False, # Enable sampling
        num_return_sequences=1,  # Return one sequence
        output_attentions=True,
    ) 
# beam_search_output, selected_regions, _, _ = output
# selected_regions = selected_regions.detach().cpu().numpy()

#                 # generated_sentences_for_selected_regions is a List[str] of length "num_regions_selected_in_batch"
# generated_sents_for_selected_regions = tokenizer.batch_decode(
#     beam_search_output, skip_special_tokens=True, clean_up_tokenization_spaces=True
# )

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Pretrained GPT-2 모델 및 토크나이저 불러오기
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Fine-tuning 가능하도록 모델 학습 모드로 전환
model.train()

# 입력 데이터 준비
text = "The quick brown fox jumps over the lazy dog."
inputs = tokenizer(text, return_tensors="pt")

# Fine-tuning 루프 (여기서는 간단한 예로 사용)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# Fine-tuning 단계
for epoch in range(1):  # 예시로 1 epoch만
    optimizer.zero_grad()
    outputs = model(**inputs, labels=inputs["input_ids"], output_attentions=True)
    loss = outputs.loss
    loss.backward()
    optimizer.step()

# 모델 평가 모드로 전환
model.eval()

# Attention Map 추출
with torch.no_grad():
    outputs = model(**inputs, output_attentions=True)

# Attention Map은 outputs.attentions에 저장
attention_maps = outputs.attentions  # List of tensors, one for each layer

# 예시: 첫 번째 레이어, 첫 번째 헤드의 Attention Map
first_layer_first_head = attention_maps[0][0]  # Shape: (batch_size, num_heads, seq_len, seq_len)
print("Attention Map Shape:", first_layer_first_head.shape)


In [ ]:
from typing import List, Dict, Optional, Tuple

import torch
import torch.nn as nn

from src.binary_classifier.binary_classifier_region_abnormal import BinaryClassifierRegionAbnormal
from src.binary_classifier.binary_classifier_region_selection import BinaryClassifierRegionSelection
from src.object_detector.object_detector import ObjectDetector
from src.language_model.language_model import LanguageModel


class ReportGenerationModel(nn.Module):
    """
    Full model consisting of:
        - object detector encoder
        - binary classifier for selecting regions for sentence genneration
        - binary classifier for detecting if a region is abnormal or normal (to encode this information in the region feature vectors)
        - language model decoder
    """

    def __init__(self, pretrain_without_lm_model=False):
        super().__init__()
        self.pretrain_without_lm_model = pretrain_without_lm_model

        self.object_detector = ObjectDetector(return_feature_vectors=True)
        # Load the best object detector from the 1st training stage here when starting the 2nd training stage
        # path_to_best_object_detector_weights = "/u/home/tanida/runs/object_detector/run_10/weights/val_loss_13.482_epoch_6.pth"
        # self.object_detector.load_state_dict(torch.load(path_to_best_object_detector_weights))

        self.binary_classifier_region_selection = BinaryClassifierRegionSelection()
        self.binary_classifier_region_abnormal = BinaryClassifierRegionAbnormal()

        self.language_model = LanguageModel()

    def forward(
        self,
        images: torch.FloatTensor,  # images is of shape [batch_size x 1 x 512 x 512] (whole gray-scale images of size 512 x 512)
        image_targets: List[Dict],  # contains a dict for every image with keys "boxes" and "labels"
        input_ids: torch.LongTensor,  # shape [(batch_size * 29) x seq_len], 1 sentence for every region for every image (sentence can be empty, i.e. "")
        attention_mask: torch.FloatTensor,  # shape [(batch_size * 29) x seq_len]
        region_has_sentence: torch.BoolTensor,  # shape [batch_size x 29], ground truth boolean mask that indicates if a region has a sentence or not
        region_is_abnormal: torch.BoolTensor,  # shape [batch_size x 29], ground truth boolean mask that indicates if a region has is abnormal or not
        return_loss: bool = True,
        past_key_values: Optional[Tuple[Tuple[torch.Tensor]]] = None,
        position_ids: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = False,
    ):
        """
        Forward method is used for training and evaluation of model.
        Generate method is used for inference.
        """
        if self.training:
            # top_region_features of shape [batch_size x 29 x 1024] (i.e. 1 feature vector for every region for every image in batch)
            # class_detected is a boolean tensor of shape [batch_size x 29]. Its value is True for a class if the object detector detected the class/region in the image
            obj_detector_loss_dict, top_region_features, class_detected = self.object_detector(images, image_targets)

            # delete tensors that we don't need anymore to free up GPU resources
            del images
            del image_targets

            # during training, only get the two losses for the two binary classifiers

            classifier_loss_region_selection = self.binary_classifier_region_selection(
                top_region_features, class_detected, return_loss=True, region_has_sentence=region_has_sentence
            )

            classifier_loss_region_abnormal = self.binary_classifier_region_abnormal(
                top_region_features, class_detected, region_is_abnormal
            )

            if self.pretrain_without_lm_model:
                return obj_detector_loss_dict, classifier_loss_region_selection, classifier_loss_region_abnormal

            # to train the decoder, we want to use only the top region features (and corresponding input_ids, attention_mask)
            # of regions that were both detected by the object detector and have a sentence as the ground truth
            # this is done under the assumption that at inference time, the binary classifier for region selection will do an adequate job
            # at selecting those regions that need a sentence to be generated by itself
            valid_input_ids, valid_attention_mask, valid_region_features = self.get_valid_decoder_input_for_training(
                class_detected, region_has_sentence, input_ids, attention_mask, top_region_features
            )

            del top_region_features
            del region_has_sentence
            del region_is_abnormal
            del class_detected
            del input_ids
            del attention_mask

        else:
            # during evaluation, also return detections (i.e. detected bboxes)
            obj_detector_loss_dict, detections, top_region_features, class_detected = self.object_detector(images, image_targets)

            del images
            del image_targets

            # during evaluation, for the binary classifier for region selection, get the loss, the regions that were selected by the classifier
            # (and that were also detected) and the corresponding region features (selected_region_features)
            # this is done to evaluate the decoder under "real-word" conditions, i.e. the binary classifier decides which regions get a sentence
            classifier_loss_region_selection, selected_regions, selected_region_features = self.binary_classifier_region_selection(
                top_region_features, class_detected, return_loss=True, region_has_sentence=region_has_sentence
            )

            # for the binary classifier for abnormal/normal detection, get the loss and the predicted abnormal regions
            classifier_loss_region_abnormal, predicted_abnormal_regions = self.binary_classifier_region_abnormal(
                top_region_features, class_detected, region_is_abnormal
            )

            if self.pretrain_without_lm_model:
                return obj_detector_loss_dict, classifier_loss_region_selection, classifier_loss_region_abnormal, detections, class_detected, selected_regions, predicted_abnormal_regions

            del top_region_features
            del region_has_sentence
            del region_is_abnormal

            # use the selected_regions mask to filter the inputs_ids and attention_mask to those that correspond to regions that were selected
            valid_input_ids, valid_attention_mask = self.get_valid_decoder_input_for_evaluation(selected_regions, input_ids, attention_mask)
            valid_region_features = selected_region_features

            del input_ids
            del attention_mask

        # valid_input_ids can be empty if during:
        # training:
        #   - the regions that have a gt sentence (specified by region_has_sentence) were all not detected (specified by class_detected).
        #   This can happend if e.g. a lateral chest x-ray was erroneously included in the dataset (and hence the object detector not detecting
        #   any regions, since it was trained on frontal chest x-rays)
        # evaluation:
        #   - no regions were selected by the binary classifier (specified by selected_regions)
        #   - the regions that were selected by the binary classifier for region selection were all not detected (also specified by selected_regions,
        #   since class_detected is encoded in selected_regions). Again, the reason might be a bad input image
        #
        # empty valid_input_ids (and thus empty valid_attention_mask, valid_region_features) will throw an exception in the language model,
        # which is why we have to return early
        if valid_input_ids.shape[0] == 0:
            return -1

        language_model_loss = self.language_model(
            valid_input_ids,
            valid_attention_mask,
            valid_region_features,
            return_loss,
            past_key_values,
            position_ids,
            use_cache,
        )

        del valid_input_ids
        del valid_attention_mask
        del valid_region_features

        if self.training:
            return obj_detector_loss_dict, classifier_loss_region_selection, classifier_loss_region_abnormal, language_model_loss
        else:
            # class_detected needed to evaluate how good the object detector is at detecting the different regions during evaluation
            # detections and class_detected needed to compute IoU of object detector during evaluation
            # selected_regions needed to evaluate binary classifier for region selection during evaluation and
            # to map each generated sentence to its corresponding region (for example for plotting)
            # predicted_abnormal_regions needed to evalute the binary classifier for normal/abnormal detection
            return (
                obj_detector_loss_dict,
                classifier_loss_region_selection,
                classifier_loss_region_abnormal,
                language_model_loss,
                detections,
                class_detected,
                selected_regions,
                predicted_abnormal_regions
            )

    def get_valid_decoder_input_for_training(
        self,
        class_detected,  # shape [batch_size x 29]
        region_has_sentence,  # shape [batch_size x 29]
        input_ids,  # shape [(batch_size * 29) x seq_len]
        attention_mask,  # shape [(batch_size * 29) x seq_len]
        region_features,  # shape [batch_size x 29 x 1024]
    ):
        """
        We want to train the decoder only on region features (and corresponding input_ids/attention_mask) whose corresponding sentences are non-empty and
        that were detected by the object detector.
        """
        # valid is of shape [batch_size x 29]
        valid = torch.logical_and(class_detected, region_has_sentence)

        # reshape to [(batch_size * 29)], such that we can apply the mask to input_ids and attention_mask
        valid_reshaped = valid.reshape(-1)

        valid_input_ids = input_ids[valid_reshaped]  # of shape [num_detected_regions_with_non_empty_gt_phrase_in_batch x seq_len]
        valid_attention_mask = attention_mask[valid_reshaped]  # of shape [num_detected_regions_with_non_empty_gt_phrase_in_batch x seq_len]
        valid_region_features = region_features[valid]  # of shape [num_detected_regions_with_non_empty_gt_phrase_in_batch x 1024]

        return valid_input_ids, valid_attention_mask, valid_region_features

    def get_valid_decoder_input_for_evaluation(
        self,
        selected_regions,  # shape [batch_size x 29]
        input_ids,  # shape [(batch_size * 29) x seq_len]
        attention_mask  # shape [(batch_size * 29) x seq_len]
    ):
        """
        For evaluation, we want to evaluate the decoder on the top_region_features selected by the classifier to get a sentence generated.
        We also have to get the corresponding input_ids and attention_mask accordingly.
        """
        # reshape to [(batch_size * 29)]
        selected_regions = selected_regions.reshape(-1)

        valid_input_ids = input_ids[selected_regions]  # of shape [num_regions_selected_in_batch x seq_len]
        valid_attention_mask = attention_mask[selected_regions]  # of shape [num_regions_selected_in_batch x seq_len]

        return valid_input_ids, valid_attention_mask

    @torch.no_grad()
    def generate(
        self,
        images: torch.FloatTensor,  # images is of shape [batch_size x 1 x 512 x 512] (whole gray-scale images of size 512 x 512)
        max_length: int = None,
        num_beams: int = 1,
        num_beam_groups: int = 1,
        do_sample: bool = False,
        num_return_sequences: int = 1,
        early_stopping: bool = False,
    ):
        """
        In inference mode, we usually input 1 image (with 29 regions) at a time.

        The object detector first finds the region features for all 29 regions.

        The binary classifier takes the region_features of shape [batch_size=1, 29, 1024] and returns:
            - selected_region_features: shape [num_regions_selected_in_batch, 1024],
            all region_features which were selected by the classifier to get a sentence generated (and which were also detected by the object detector)

            - selected_regions: shape [batch_size x 29], boolean matrix that indicates which regions were selected to get a sentences generated
            (these regions must also have been detected by the object detector).
            This is needed in case we want to find the corresponding reference sentences to compute scores for metrics such as BertScore or BLEU.

        The decoder then takes the selected_region_features and generates output ids for the batch.
        These output ids can then be decoded by the tokenizer to get the generated sentences.

        We also return selected_regions, such that we can map each generated sentence to a selected region.
        We also return detections, such that we can map each generated sentence to a bounding box.
        We also return class_detected to know which regions were not detected by the object detector (can be plotted).
        """
        # top_region_features of shape [batch_size x 29 x 1024]
        _, detections, top_region_features, class_detected = self.object_detector(images)

        del images

        # selected_region_features is of shape [num_regions_selected_in_batch x 1024]
        # selected_regions is of shape [batch_size x 29] and is True for regions that should get a sentence
        # (it has exactly num_regions_selected_in_batch True values)
        selected_regions, selected_region_features = self.binary_classifier_region_selection(
            top_region_features, class_detected, return_loss=False
        )

        del top_region_features

        # selected_region_features can be empty if no region was both detected by the object detector and selected
        # by the binary classifier to get a sentence generated. This can happen especially early on in training
        # Since this would throw an exception in the language model, we return early
        if selected_region_features.shape[0] == 0:
            return -1

        # output_ids of shape (num_regions_selected_in_batch x longest_generated_sequence_length)
        output_ids = self.language_model.generate(
            selected_region_features,
            max_length,
            num_beams,
            num_beam_groups,
            do_sample,
            num_return_sequences,
            early_stopping,
        )

        del selected_region_features

        return output_ids, selected_regions, detections, class_detected


In [ ]:
import matplotlib.pyplot as plt
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Pretrained GPT-2 모델 및 토크나이저 불러오기
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# 입력 텍스트
text = "The quick brown fox jumps over the lazy dog"
inputs = tokenizer(text, return_tensors="pt")

# 모델 평가 모드 및 Attention Map 추출
model.eval()
with torch.no_grad():
    outputs = model(**inputs, output_attentions=True)

# Attention Map 가져오기 (첫 번째 레이어의 첫 번째 헤드)
attention_maps = outputs.attentions  # List of attention maps, one per layer
layer_idx = 0  # 첫 번째 레이어
head_idx = 0   # 첫 번째 헤드

# Attention Map 크기: (batch_size, num_heads, seq_len, seq_len)
attention_map = attention_maps[layer_idx][0, head_idx].numpy()  # (seq_len, seq_len)

# 히트맵 시각화
plt.figure(figsize=(10, 8))
plt.imshow(attention_map, cmap="viridis")
plt.colorbar()
plt.xticks(range(len(text.split())), text.split(), rotation=90)  # X축: 입력 토큰
plt.yticks(range(len(text.split())), text.split())              # Y축: 입력 토큰
plt.title(f"Attention Map (Layer {layer_idx + 1}, Head {head_idx + 1})")
plt.xlabel("Key Tokens")
plt.ylabel("Query Tokens")
plt.show()
